In [1]:
import models.models_multi_task as md_multi
from models.multitask_training_session import *
from datasets.iemocap import IemocapDataset
from datasets.ramas import RamasDataset
from constants import *
from torchsummary import summary
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import os
import pickle
import pandas as pd 

# Проба многозадачного обучения

## Модель:

Подготовил следующую архитектуру, основанную на однозадачной AlexNet-like архитектуре: 

In [2]:
model = md_multi.AlexNetMultiTask(num_emotions=4, num_speakers=10, num_genders=2)
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 
device = torch.device("cpu")
model = model.to(device)
print(model)
summary(model, (1, 224, 224), batch_size=32,
        device='cpu'
       )
model=None
torch.cuda.empty_cache()

AlexNetMultiTask(
  (features): Sequential(
    (0): Conv2d(1, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(4, 4), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(5, 5))
  (joint_classifier): Sequential(
    (0): Linear(in_features=6400, out_features=2048, bias=True)
    (1): Dropout(p=0.5, i

## Проверяю модель и свой код для обучения на работоспособность

Мой код для обучения построен вокруг класса TrainingSession в модуле multitask_training_session.py <br>
У этого класса есть метод overfit_one_batch, этот метод позволяет обучить модель на малой выборке данных, это полезно для того, чтобы проверить на работоспособность свой код. <br>
На этой стадии я убрал из модели регуляризацию.

In [4]:
train_ds = ds.IemocapDataset(  # Без препроцессинга, тренировочный
    PATH_TO_PICKLE, IEMOCAP_PATH_TO_WAVS, IEMOCAP_PATH_TO_EGEMAPS, IEMOCAP_PATH_FOR_PARSER, 
    base_name='IEMOCAP-4', label_type='four', mode='train', preprocessing=False, 
    augmentation=True, padding='repeat', spectrogram_shape=224, spectrogram_type='melspec', tasks=('emotion', 'speaker', 'gender') 
)
test_ds = ds.IemocapDataset(  # Без препроцессинга, тестовый
    PATH_TO_PICKLE, IEMOCAP_PATH_TO_WAVS, IEMOCAP_PATH_TO_EGEMAPS, IEMOCAP_PATH_FOR_PARSER, 
    base_name='IEMOCAP-4', label_type='four', mode='test', preprocessing=False, 
    augmentation=True, padding='repeat', spectrogram_shape=224, spectrogram_type='melspec', tasks=('emotion', 'speaker', 'gender') 
)

============= INITIALIZING DATASET IEMOCAP-4_four_prep-false_224_train ===============
=========================== SUCCESS! ====================================
============= INITIALIZING DATASET IEMOCAP-4_four_prep-false_224_test ===============
=========================== SUCCESS! ====================================


In [5]:
model = md_multi.AlexNetMultiTask(4, 10, 2)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 
# device = torch.device("cpu") 
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.001)

In [5]:
# Инициализируем экземпляр класса TrainingSession
ts = TrainingSession(name='FirstTry',
                      model=model,
                      train_dataset=train_ds,
                      test_dataset=test_ds,
                      criterion=criterion,
                      optimizer=optimizer,
                      num_epochs=200,
                      batch_size=32,
                      device=device,
                     path_to_weights=WEIGHTS_FOLDER,
                     path_to_results=RESULTS_FOLDER)

INITIALIZING TRAINING SESSION...
Loaders ready
TRAINING SESSION FirstTry__IEMOCAP-4_four_prep-false_224_train INITIALIZED
Trying to load checkpoint from file
Found file
Loading file models\training_sessions\FirstTry__IEMOCAP-4_four_prep-false_224_train.pt
Updating model...
Updating optimizer...
Success!


In [9]:
ts.overfit_one_batch(num_epochs=5, batch_size=32)  # 5 эпох на одном батче размером 32, в одной эпохе 50 одинаковых батчей

TRAIN SIZE 2858
TEST SIZE 714
Epoch 0
Emotion loss | Speaker loss | Gender loss | Total loss
0: 1.4414201974868774 | 2.3414275646209717 | 0.44765982031822205 | 4.230507850646973
1: 0.9333392381668091 | 1.9413270950317383 | 0.43485891819000244 | 3.3095250129699707
2: 0.8018255233764648 | 1.745065689086914 | 0.32746991515159607 | 2.874361038208008
3: 0.8215715289115906 | 1.4112733602523804 | 0.10227905958890915 | 2.3351237773895264
4: 0.5799142122268677 | 1.30913245677948 | 0.06190920248627663 | 1.950955867767334
5: 0.6207345724105835 | 1.1703191995620728 | 0.047992024570703506 | 1.839045763015747
6: 0.5136659741401672 | 1.0797524452209473 | 0.028099965304136276 | 1.6215183734893799
7: 0.46535301208496094 | 1.0105384588241577 | 0.07338234037160873 | 1.5492738485336304
8: 0.36521920561790466 | 1.0084718465805054 | 0.014499970711767673 | 1.3881911039352417
9: 0.3800300657749176 | 0.7452290654182434 | 0.012217322364449501 | 1.1374764442443848
10: 0.21810980141162872 | 0.6663102507591248 | 0

36: 0.012218687683343887 | 0.03701668605208397 | 0.12943458557128906 | 0.17866995930671692
37: 0.0037573757581412792 | 0.366527259349823 | 1.3485373528965283e-06 | 0.3702859878540039
38: 0.09070053696632385 | 0.0444038026034832 | 0.00030247398535721004 | 0.13540682196617126
39: 0.0009722106624394655 | 0.011130941100418568 | 0.011927219107747078 | 0.024030370637774467
40: 0.013412228785455227 | 0.04771655797958374 | 0.0013015508884564042 | 0.06243033707141876
41: 0.005402492359280586 | 0.012367974035441875 | 0.00048231796245090663 | 0.01825278252363205
42: 0.005212982185184956 | 0.0824459046125412 | 0.01985791139304638 | 0.1075168028473854
43: 0.03840407729148865 | 0.08887898176908493 | 0.009803637862205505 | 0.13708670437335968
44: 0.11791308224201202 | 0.05791176110506058 | 0.003193353768438101 | 0.17901819944381714
45: 0.022144043818116188 | 0.01419439073652029 | 4.3993472900183406e-06 | 0.03634283319115639
46: 0.017170283943414688 | 0.016948655247688293 | 0.0006145340739749372 | 0.0

16: 0.005717601161450148 | 0.0010633096098899841 | 1.110121161218558e-06 | 0.006782020907849073
17: 0.0003339503309689462 | 0.002387040061876178 | 8.940686058167557e-08 | 0.002721079858019948
18: 0.0005621638265438378 | 0.007715967018157244 | 2.2351736461700966e-08 | 0.008278152905404568
19: 0.0858149379491806 | 0.0020198433194309473 | 6.183922778291162e-07 | 0.0878354012966156
20: 0.007901272736489773 | 0.0037698964588344097 | 8.456351565655496e-07 | 0.011672014370560646
21: 0.00012759340461343527 | 0.0006163700018078089 | 0.00032811093842610717 | 0.0010720742866396904
22: 7.725906471023336e-05 | 0.0027982445899397135 | 6.46609187242575e-05 | 0.0029401646461337805
23: 0.04838520288467407 | 0.062128085643053055 | 0.0020728365052491426 | 0.1125861182808876
24: 0.0006428712513297796 | 0.01046313438564539 | 2.188776488765143e-05 | 0.011127893812954426
25: 0.008424758911132812 | 0.023545067757368088 | 1.0765209481178317e-05 | 0.031980592757463455
26: 0.00012566197256091982 | 0.003842133795

# Time passed: 7 s
# Epoch losses | emotion = 0.0288 | speaker = 0.0210 | gender = 0.0161 |
# Train accuracies | emotion = 0.99625 | speaker = 0.995 | gender = 0.9975 |


Модель переобучается, а значит наш код работает нормально. Можно попробовать обучить модель на всем датасете. 

## Обучаем модель на IEMOCAP

Для этого у класса TrainingSession есть метод execute().  <br>
Он обучает модель, сохраняет ее после каждой эпохи, а так же копию модели в случае, если на этой эпохе был достигнут лучший результат. На этом этапе регуляризация включена.

In [10]:
ts.execute()

=============TRAINING SESSION STARTED AT EPOCH 1=====================
Epoch #1
# Time passed: 68 s
# Epoch losses | emotion = 1.3595 | speaker = 2.4053 | gender = 0.7198 | total = 16019.1389 |
# Train accuracies | emotion = 0.4008958566629339 | speaker = 0.1187010078387458 | gender = 0.5663493840985442 |
# Validation process on validation set
# Validation losses | emotion = 0.0371 | speaker = 0.0726 | gender = 0.0213 | total = 7401.1727 |
# Validation accuracies | emotion = 0.4322508398656215 | speaker = 0.14669652855543114 | gender = 0.5856662933930571 |
# Saving checkpoint...
## Saving best model_alex
# Done and done!
Epoch #2
# Time passed: 66 s
# Epoch losses | emotion = 1.2164 | speaker = 2.2841 | gender = 0.6160 | total = 14704.1351 |
# Train accuracies | emotion = 0.4209126539753639 | speaker = 0.12234042553191489 | gender = 0.618421052631579 |
# Validation process on validation set
# Validation losses | emotion = 0.0356 | speaker = 0.0739 | gender = 0.0177 | total = 7132.9266 |

# Time passed: 60 s
# Epoch losses | emotion = 0.9956 | speaker = 1.9256 | gender = 0.1356 | total = 10918.8157 |
# Train accuracies | emotion = 0.5290753479443289 | speaker = 0.18267077267637177 | gender = 0.8722804351303791 |
# Validation process on validation set
# Validation losses | emotion = 0.0309 | speaker = 0.0647 | gender = 0.0089 | total = 5395.1400 |
# Validation accuracies | emotion = 0.606942889137738 | speaker = 0.3023516237402016 | gender = 0.9384098544232923 |
# Saving checkpoint...
## Saving best model_alex
# Done and done!
Epoch #15
# Time passed: 63 s
# Epoch losses | emotion = 1.0048 | speaker = 1.8686 | gender = 0.1515 | total = 10805.0271 |
# Train accuracies | emotion = 0.5326241134751774 | speaker = 0.19010824934677117 | gender = 0.8770250093318402 |
# Validation process on validation set
# Validation losses | emotion = 0.0255 | speaker = 0.0617 | gender = 0.0066 | total = 5212.1861 |
# Validation accuracies | emotion = 0.606942889137738 | speaker = 0.318029115

# Validation losses | emotion = 0.0277 | speaker = 0.0397 | gender = 0.0014 | total = 4642.8207 |
# Validation accuracies | emotion = 0.6013437849944009 | speaker = 0.41993281075027994 | gender = 0.9406494960806271 |
# Saving checkpoint...
# Done and done!
Epoch #28
# Time passed: 53 s
# Epoch losses | emotion = 0.9255 | speaker = 1.3175 | gender = 0.0967 | total = 8357.1652 |
# Train accuracies | emotion = 0.5692889137737962 | speaker = 0.294432890737482 | gender = 0.9148636218205087 |
# Validation process on validation set
# Validation losses | emotion = 0.0356 | speaker = 0.0522 | gender = 0.0060 | total = 4493.1566 |
# Validation accuracies | emotion = 0.6349384098544233 | speaker = 0.46472564389697646 | gender = 0.9552071668533034 |
# Saving checkpoint...
# Done and done!
Epoch #29
# Time passed: 50 s
# Epoch losses | emotion = 0.9132 | speaker = 1.2896 | gender = 0.0889 | total = 8185.9788 |
# Train accuracies | emotion = 0.571485114105881 | speaker = 0.30171448430320114 | gender

# Time passed: 50 s
# Epoch losses | emotion = 0.8451 | speaker = 1.0786 | gender = 0.0720 | total = 7128.6494 |
# Train accuracies | emotion = 0.5925354382323218 | speaker = 0.37378799879824104 | gender = 0.9319367437795318 |
# Validation process on validation set
# Validation losses | emotion = 0.0316 | speaker = 0.0323 | gender = 0.0159 | total = 4060.5910 |
# Validation accuracies | emotion = 0.6506159014557671 | speaker = 0.5531914893617021 | gender = 0.9540873460246361 |
# Saving checkpoint...
# Done and done!
Epoch #42
# Time passed: 50 s
# Epoch losses | emotion = 0.8552 | speaker = 1.0710 | gender = 0.0716 | total = 7136.1463 |
# Train accuracies | emotion = 0.5938649816029435 | speaker = 0.3787127392950461 | gender = 0.9329107342825148 |
# Validation process on validation set
# Validation losses | emotion = 0.0177 | speaker = 0.0402 | gender = 0.0066 | total = 4159.9014 |
# Validation accuracies | emotion = 0.6427771556550952 | speaker = 0.5464725643896976 | gender = 0.947368

# Done and done!
Epoch #55
# Time passed: 49 s
# Epoch losses | emotion = 0.7826 | speaker = 0.9457 | gender = 0.0625 | total = 6396.6078 |
# Train accuracies | emotion = 0.6123282093046931 | speaker = 0.43357426448131936 | gender = 0.942502290542604 |
# Validation process on validation set
# Validation losses | emotion = 0.0346 | speaker = 0.0322 | gender = 0.0056 | total = 4022.4774 |
# Validation accuracies | emotion = 0.6349384098544233 | speaker = 0.5666293393057111 | gender = 0.9652855543113102 |
# Saving checkpoint...
# Done and done!
Epoch #56
# Time passed: 50 s
# Epoch losses | emotion = 0.7734 | speaker = 0.9358 | gender = 0.0637 | total = 6332.6761 |
# Train accuracies | emotion = 0.6136018237082067 | speaker = 0.43717505199168133 | gender = 0.9430741081426972 |
# Validation process on validation set
# Validation losses | emotion = 0.0252 | speaker = 0.0387 | gender = 0.0022 | total = 4041.9369 |
# Validation accuracies | emotion = 0.6528555431131019 | speaker = 0.574468085

# Done and done!
Epoch #69
# Time passed: 60 s
# Epoch losses | emotion = 0.7293 | speaker = 0.8020 | gender = 0.0528 | total = 5658.1953 |
# Train accuracies | emotion = 0.629854585585147 | speaker = 0.4807155492802311 | gender = 0.9499732216758362 |
# Validation process on validation set
# Validation losses | emotion = 0.0301 | speaker = 0.0358 | gender = 0.0179 | total = 4090.4267 |
# Validation accuracies | emotion = 0.6338185890257558 | speaker = 0.5621500559910414 | gender = 0.9507278835386338 |
# Saving checkpoint...
# Done and done!
Epoch #70
# Time passed: 61 s
# Epoch losses | emotion = 0.7049 | speaker = 0.8028 | gender = 0.0546 | total = 5580.5698 |
# Train accuracies | emotion = 0.6310990241561351 | speaker = 0.48376259798432253 | gender = 0.9503799392097264 |
# Validation process on validation set
# Validation losses | emotion = 0.0369 | speaker = 0.0370 | gender = 0.0094 | total = 4085.2251 |
# Validation accuracies | emotion = 0.6371780515117581 | speaker = 0.5946248600

# Done and done!
Epoch #83
# Time passed: 61 s
# Epoch losses | emotion = 0.6674 | speaker = 0.7450 | gender = 0.0449 | total = 5205.6116 |
# Train accuracies | emotion = 0.6457352365790149 | speaker = 0.5191583804422618 | gender = 0.9551599454930585 |
# Validation process on validation set
# Validation losses | emotion = 0.0274 | speaker = 0.0347 | gender = 0.0253 | total = 4431.4551 |
# Validation accuracies | emotion = 0.6472564389697648 | speaker = 0.574468085106383 | gender = 0.948488241881299 |
# Saving checkpoint...
# Done and done!
Epoch #84
# Time passed: 61 s
# Epoch losses | emotion = 0.6745 | speaker = 0.7326 | gender = 0.0411 | total = 5173.0467 |
# Train accuracies | emotion = 0.6468265077587586 | speaker = 0.5215565509518477 | gender = 0.9555271156614942 |
# Validation process on validation set
# Validation losses | emotion = 0.0323 | speaker = 0.0384 | gender = 0.0077 | total = 4479.6075 |
# Validation accuracies | emotion = 0.6349384098544233 | speaker = 0.557670772676

# Done and done!
Epoch #97
# Time passed: 60 s
# Epoch losses | emotion = 0.6085 | speaker = 0.6316 | gender = 0.0422 | total = 4580.3048 |
# Train accuracies | emotion = 0.6609309520785953 | speaker = 0.550917214070491 | gender = 0.9594699899562462 |
# Validation process on validation set
# Validation losses | emotion = 0.0213 | speaker = 0.0500 | gender = 0.0053 | total = 4519.2555 |
# Validation accuracies | emotion = 0.6405375139977604 | speaker = 0.5722284434490481 | gender = 0.9451287793952967 |
# Saving checkpoint...
# Done and done!
Epoch #98
# Time passed: 58 s
# Epoch losses | emotion = 0.6135 | speaker = 0.6951 | gender = 0.0582 | total = 4882.4820 |
# Train accuracies | emotion = 0.661976940832324 | speaker = 0.5528344036382751 | gender = 0.9596978769111227 |
# Validation process on validation set
# Validation losses | emotion = 0.0324 | speaker = 0.0395 | gender = 0.0003 | total = 4662.7024 |
# Validation accuracies | emotion = 0.6338185890257558 | speaker = 0.556550951847

In [6]:
ts.create_results_file()

In [18]:
results = pd.read_csv(os.path.join(ts.path_to_results, f'{ts.name}_results.csv'), delimiter=';')
results_test = results.loc[results['subset']=='test']
results_test_accuracy = results_test.loc[results_test['metric']=='accuracy']
results_test_accuracy.drop('epoch', axis=1, inplace=True)
results_test_accuracy.groupby('task').max()

,result,subset,metric
task,,,
emotion,0.653975,test,accuracy
gender,0.965286,test,accuracy
speaker,0.603583,test,accuracy


### К сожалению, при обычном суммировании потерь, модель не опережает по точности однозадачную. Нужно пробовать взвешенную сумму потерь.

В класс TrainingSession я добавил параметр loss_weighter. Этот параметр принимает функцию от отдельных потерь, и возвращает вычисленную определенным образом сумму. В файле multitask_training_session.py на данный момент есть следующие функции для взвешивания потерь: <br>
1. Невзвешенная сумма unweighted_sum(loss_1, loss_2, loss_3). Параметр по умолчанию.
2. Усредненная сумма averaged_sum(loss_1, loss_2, loss_3). Подсчитывает итоговый лосс по формуле loss = a x loss_1 + b x loss_2 + c x loss_3, где a = loss_1 / (loss_1 + loss_2 + loss_3), b и с - аналогично. 
3. Автоматическое взвешенное суммивание функций потерь с помощью оценки алеаторической неопределенности модели по отношению к отдельной задаче. Описан в классе AutomaticWeightedLoss(nn.Module), добро пожаловать в исходный код. <br> Является имплементацией метода, описанного в статье [Liebel L, Körner M. Auxiliary tasks in multi-task learning[J]. arXiv preprint arXiv:1805.06334, 2018.], https://arxiv.org/pdf/1805.06334.pdf (это более практическая статья)<br> Метод является небольшим усовершенствованием другого метода, описанного в статье [Multi-Task Learning Using Uncertainty to Weigh Losses for Scene Geometry and Semantics], https://openaccess.thecvf.com/content_cvpr_2018/html/Kendall_Multi-Task_Learning_Using_CVPR_2018_paper.html (в этой статье описана вся теория метода)

#### Пробуем усредненную сумму

In [2]:
train_ds = ds.IemocapDataset(  # Без препроцессинга, тренировочный
    PATH_TO_PICKLE, IEMOCAP_PATH_TO_WAVS, IEMOCAP_PATH_TO_EGEMAPS, IEMOCAP_PATH_FOR_PARSER, 
    base_name='IEMOCAP-4', label_type='four', mode='train', preprocessing=False, 
    augmentation=True, padding='repeat', spectrogram_shape=224, spectrogram_type='melspec', tasks=('emotion', 'speaker', 'gender') 
)
test_ds = ds.IemocapDataset(  # Без препроцессинга, тестовый
    PATH_TO_PICKLE, IEMOCAP_PATH_TO_WAVS, IEMOCAP_PATH_TO_EGEMAPS, IEMOCAP_PATH_FOR_PARSER, 
    base_name='IEMOCAP-4', label_type='four', mode='test', preprocessing=False, 
    augmentation=True, padding='repeat', spectrogram_shape=224, spectrogram_type='melspec', tasks=('emotion', 'speaker', 'gender') 
)

============= INITIALIZING DATASET IEMOCAP-4_four_prep-false_224_train ===============
=========================== SUCCESS! ====================================
============= INITIALIZING DATASET IEMOCAP-4_four_prep-false_224_test ===============
=========================== SUCCESS! ====================================


##### Снова попробуем заоверфитить 1 батч

In [3]:
model = md_multi.AlexNetMultiTask(4, 10, 2)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 
# device = torch.device("cpu") 
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model.parameters(), lr=1e-5)
ts = TrainingSession(name='AverageWeighting',
                      model=model,
                      train_dataset=train_ds,
                      test_dataset=test_ds,
                      criterion=criterion,
                      optimizer=optimizer,
                      num_epochs=300,
                      batch_size=32,
                      device=device,
                     path_to_weights=WEIGHTS_FOLDER,
                     path_to_results=RESULTS_FOLDER,
                    loss_weighter=averaged_sum  # Из файла multitask_training_session.py
                    )

INITIALIZING TRAINING SESSION...
Loaders ready
TRAINING SESSION AverageWeighting__IEMOCAP-4_four_prep-false_224_train INITIALIZED
Trying to load checkpoint from file
File not found, starting from scratch...


In [4]:
ts.overfit_one_batch(num_epochs=15, batch_size=32)  # 15 эпох на одном батче размером 32, в одной эпохе 50 одинаковых батчей

TRAIN SIZE 2858
TEST SIZE 714
Epoch 0
Emotion loss | Speaker loss | Gender loss | Total loss
0: 1.3885794878005981 | 2.3076436519622803 | 0.6935040950775146 | 1.7619134187698364
1: 1.3848907947540283 | 2.3031997680664062 | 0.6924958825111389 | 1.7582582235336304
2: 1.383169412612915 | 2.298151731491089 | 0.6914010047912598 | 1.7546722888946533
3: 1.3858224153518677 | 2.301992177963257 | 0.6912910342216492 | 1.757791519165039
4: 1.3843141794204712 | 2.3024511337280273 | 0.6935303211212158 | 1.7575504779815674
5: 1.3844422101974487 | 2.299452781677246 | 0.69172203540802 | 1.7557849884033203
6: 1.3862930536270142 | 2.295719623565674 | 0.6915339827537537 | 1.7538068294525146
7: 1.387851357460022 | 2.297340154647827 | 0.6952111124992371 | 1.754912257194519
8: 1.3870453834533691 | 2.3033125400543213 | 0.6934806704521179 | 1.7587460279464722
9: 1.3837106227874756 | 2.2962496280670166 | 0.6934875845909119 | 1.7533860206604004
10: 1.3859975337982178 | 2.298274517059326 | 0.6920468807220459 | 1.

49: 1.4101866483688354 | 2.0443553924560547 | 0.8206737637519836 | 1.6002748012542725
# Time passed: 1 s
# Epoch losses | emotion = 1.3881 | speaker = 2.1182 | gender = 0.7922 |
# Train accuracies | emotion = 0.300625 | speaker = 0.181875 | gender = 0.44125 |
Epoch 2
Emotion loss | Speaker loss | Gender loss | Total loss
0: 1.368632197380066 | 2.202056407928467 | 0.8588191866874695 | 1.6841096878051758
1: 1.3839476108551025 | 2.0836105346679688 | 0.7997249960899353 | 1.616087794303894
2: 1.3413037061691284 | 2.031942367553711 | 0.7671299576759338 | 1.5738604068756104
3: 1.3559154272079468 | 2.082592010498047 | 0.8357757925987244 | 1.6082736253738403
4: 1.3196876049041748 | 2.1732399463653564 | 0.7975726127624512 | 1.6549746990203857
5: 1.3774828910827637 | 2.1159303188323975 | 0.8137949109077454 | 1.633745551109314
6: 1.3274164199829102 | 2.060295343399048 | 0.7823294997215271 | 1.5872483253479004
7: 1.3924083709716797 | 2.072855234146118 | 0.7882145047187805 | 1.6120482683181763
8: 1.

39: 1.3697870969772339 | 1.948333978652954 | 0.7717559337615967 | 1.5325469970703125
40: 1.3983668088912964 | 1.9881055355072021 | 0.8177060484886169 | 1.5643095970153809
41: 1.2650234699249268 | 2.026191473007202 | 0.8082196116447449 | 1.5511786937713623
42: 1.3383331298828125 | 1.990349531173706 | 0.7328536510467529 | 1.548601508140564
43: 1.4038654565811157 | 2.0566699504852295 | 0.7655802965164185 | 1.6059291362762451
44: 1.4575566053390503 | 2.083014726638794 | 0.8611347675323486 | 1.636859655380249
45: 1.3557006120681763 | 2.0083742141723633 | 0.8385564684867859 | 1.5644171237945557
46: 1.3127737045288086 | 2.0135958194732666 | 0.7978914380073547 | 1.5553269386291504
47: 1.3662877082824707 | 2.0607919692993164 | 0.8894855380058289 | 1.5996029376983643
48: 1.306232213973999 | 1.9122782945632935 | 0.8083474040031433 | 1.4940871000289917
49: 1.4141933917999268 | 2.0063326358795166 | 0.7900732755661011 | 1.5792357921600342
# Time passed: 1 s
# Epoch losses | emotion = 1.3684 | speake

29: 1.2031354904174805 | 1.8308203220367432 | 0.7476040124893188 | 1.4169681072235107
30: 1.1797549724578857 | 1.6775882244110107 | 0.6898884177207947 | 1.3199222087860107
31: 1.139341950416565 | 1.7733644247055054 | 0.8219341039657593 | 1.3705461025238037
32: 1.1489570140838623 | 1.6128042936325073 | 0.8280462622642517 | 1.2833279371261597
33: 1.084752082824707 | 1.6184107065200806 | 0.6931372284889221 | 1.2591289281845093
34: 1.0729186534881592 | 1.523635745048523 | 0.6355739235877991 | 1.1993876695632935
35: 1.168105959892273 | 1.6071093082427979 | 0.6809796690940857 | 1.2762606143951416
36: 1.1310747861862183 | 1.540037751197815 | 0.611415445804596 | 1.2261511087417603
37: 0.9986131191253662 | 1.655480980873108 | 0.7148860096931458 | 1.2611850500106812
38: 1.0895421504974365 | 1.7624201774597168 | 0.655916690826416 | 1.346527099609375
39: 1.1325510740280151 | 1.6862497329711914 | 0.6336333751678467 | 1.3114229440689087
40: 1.0313161611557007 | 1.6898664236068726 | 0.680348157882690

19: 0.8290165662765503 | 1.0236470699310303 | 0.589945912361145 | 0.8528411388397217
20: 0.5674254298210144 | 0.9704903364181519 | 0.4741133451461792 | 0.7398533821105957
21: 0.544574499130249 | 0.9711688160896301 | 0.5958182215690613 | 0.7552371025085449
22: 0.7896404266357422 | 1.078702449798584 | 0.6543001532554626 | 0.8781422972679138
23: 0.8022997379302979 | 0.9435327649116516 | 0.6691505312919617 | 0.8205860257148743
24: 0.530787467956543 | 0.9068611264228821 | 0.5635452270507812 | 0.7104337811470032
25: 0.8359888792037964 | 1.145046353340149 | 0.602678656578064 | 0.9185343980789185
26: 0.745050311088562 | 1.1226407289505005 | 0.6313586831092834 | 0.8859512209892273
27: 0.5218392014503479 | 1.0254648923873901 | 0.6453971266746521 | 0.7937386631965637
28: 0.6936779022216797 | 1.0069772005081177 | 0.5456409454345703 | 0.7981655597686768
29: 0.7044244408607483 | 0.9218820929527283 | 0.4673302471637726 | 0.7472537755966187
30: 0.5925824046134949 | 1.0964151620864868 | 0.6217727661132

9: 0.3305553197860718 | 0.5017391443252563 | 0.43149998784065247 | 0.43298280239105225
10: 0.35076454281806946 | 0.40546560287475586 | 0.30895543098449707 | 0.3594599664211273
11: 0.3285503685474396 | 0.6034061312675476 | 0.41891467571258545 | 0.47934532165527344
12: 0.3163866698741913 | 0.468757688999176 | 0.39530447125434875 | 0.4033210873603821
13: 0.35805395245552063 | 0.47886893153190613 | 0.39269593358039856 | 0.4161681532859802
14: 0.33907073736190796 | 0.35794588923454285 | 0.34397760033607483 | 0.34718236327171326
15: 0.32486748695373535 | 0.382320374250412 | 0.4424147307872772 | 0.3892114758491516
16: 0.336997926235199 | 0.5204102396965027 | 0.466695636510849 | 0.4547975957393646
17: 0.35283005237579346 | 0.5363152623176575 | 0.32968175411224365 | 0.4273068606853485
18: 0.38380321860313416 | 0.593873143196106 | 0.33798155188560486 | 0.4668552279472351
19: 0.34604859352111816 | 0.5003461241722107 | 0.297087699174881 | 0.4008430242538452
20: 0.4321034252643585 | 0.4327074587345

8: 0.1937294751405716 | 0.16307014226913452 | 0.2682887315750122 | 0.21773213148117065
9: 0.16754990816116333 | 0.30349257588386536 | 0.1822889745235443 | 0.23481187224388123
10: 0.21686141192913055 | 0.2710193693637848 | 0.2745213210582733 | 0.2568753957748413
11: 0.22479191422462463 | 0.22226932644844055 | 0.2662913501262665 | 0.23949748277664185
12: 0.4305918216705322 | 0.4812147915363312 | 0.1422630101442337 | 0.41478830575942993
13: 0.17567284405231476 | 0.43058913946151733 | 0.21323367953300476 | 0.3193873465061188
14: 0.2849293351173401 | 0.32979273796081543 | 0.2388569414615631 | 0.28937050700187683
15: 0.3685320019721985 | 0.2563362419605255 | 0.1423804610967636 | 0.28908005356788635
16: 0.32690003514289856 | 0.285822331905365 | 0.1606796234846115 | 0.27718570828437805
17: 0.2592998147010803 | 0.2554905116558075 | 0.24711543321609497 | 0.25407058000564575
18: 0.230195090174675 | 0.41062572598457336 | 0.1274735927581787 | 0.30958548188209534
19: 0.25019171833992004 | 0.24176770

48: 0.1438210904598236 | 0.21076729893684387 | 0.13117647171020508 | 0.16945365071296692
49: 0.2675197720527649 | 0.19344067573547363 | 0.11988338828086853 | 0.21237750351428986
# Time passed: 1 s
# Epoch losses | emotion = 0.1477 | speaker = 0.2011 | gender = 0.1606 |
# Train accuracies | emotion = 0.944375 | speaker = 0.93375 | gender = 0.935 |
Epoch 13
Emotion loss | Speaker loss | Gender loss | Total loss
0: 0.10611359030008316 | 0.14944231510162354 | 0.16454018652439117 | 0.14441117644309998
1: 0.03780830278992653 | 0.15213319659233093 | 0.19189587235450745 | 0.16079621016979218
2: 0.10311079770326614 | 0.0827803835272789 | 0.14196960628032684 | 0.114804208278656
3: 0.09341743588447571 | 0.2101755142211914 | 0.18904513120651245 | 0.17992645502090454
4: 0.17665906250476837 | 0.2019353210926056 | 0.21546617150306702 | 0.19932642579078674
5: 0.14852243661880493 | 0.12186696380376816 | 0.05798689275979996 | 0.12264268845319748
6: 0.14587119221687317 | 0.14427588880062103 | 0.083128802

38: 0.08938658237457275 | 0.11627275496721268 | 0.06186322495341301 | 0.0947074294090271
39: 0.06943994760513306 | 0.07543794065713882 | 0.0851379781961441 | 0.07721756398677826
40: 0.08217129856348038 | 0.04515116289258003 | 0.039740949869155884 | 0.06207279488444328
41: 0.3816266357898712 | 0.09375032037496567 | 0.06762594729661942 | 0.29281848669052124
42: 0.052051275968551636 | 0.1716597080230713 | 0.09376072883605957 | 0.1290428787469864
43: 0.10372324287891388 | 0.09864044934511185 | 0.0571519210934639 | 0.09153511375188828
44: 0.08114024251699448 | 0.09772096574306488 | 0.11622430384159088 | 0.1004495769739151
45: 0.05289824306964874 | 0.0628921166062355 | 0.05666419863700867 | 0.05778028815984726
46: 0.11436457931995392 | 0.0806134045124054 | 0.04877996817231178 | 0.09007814526557922
47: 0.037254832684993744 | 0.08966292440891266 | 0.09399164468050003 | 0.08266644179821014
48: 0.08596985042095184 | 0.09504392743110657 | 0.054189860820770264 | 0.08231464773416519
49: 0.121880993

Работает, пробуем запускать обучение

In [4]:
ts.execute()

=============TRAINING SESSION STARTED AT EPOCH 1=====================
Epoch #1
# Time passed: 6 s
# Epoch losses | emotion = 1.3624 | speaker = 2.3018 | gender = 0.6950 | total = 1.7523 |
# Train accuracies | emotion = 0.36786114221724525 | speaker = 0.11562150055991041 | gender = 0.4916013437849944 |
# Validation process on validation set
# Validation losses | emotion = 0.0457 | speaker = 0.0743 | gender = 0.0214 | total = 0.0571 |
# Validation accuracies | emotion = 0.3807390817469205 | speaker = 0.13997760358342665 | gender = 0.49944008958566627 |
# Saving checkpoint...
## Saving best model_alex
# Done and done!
Epoch #2
# Time passed: 6 s
# Epoch losses | emotion = 1.3315 | speaker = 2.2980 | gender = 0.7255 | total = 1.7415 |
# Train accuracies | emotion = 0.37416013437849943 | speaker = 0.11506159014557671 | gender = 0.4911814109742441 |
# Validation process on validation set
# Validation losses | emotion = 0.0398 | speaker = 0.0744 | gender = 0.0253 | total = 0.0556 |
# Validati

# Validation losses | emotion = 0.0367 | speaker = 0.0757 | gender = 0.0254 | total = 0.0560 |
# Validation accuracies | emotion = 0.5274356103023516 | speaker = 0.13213885778275475 | gender = 0.40201567749160133 |
# Saving checkpoint...
## Saving best model_alex
# Done and done!
Epoch #15
# Time passed: 6 s
# Epoch losses | emotion = 1.1536 | speaker = 2.2648 | gender = 0.8129 | total = 1.6858 |
# Train accuracies | emotion = 0.4200261291526689 | speaker = 0.12530795072788353 | gender = 0.4805524449421426 |
# Validation process on validation set
# Validation losses | emotion = 0.0314 | speaker = 0.0743 | gender = 0.0329 | total = 0.0548 |
# Validation accuracies | emotion = 0.5207166853303471 | speaker = 0.11982082866741321 | gender = 0.4154535274356103 |
# Saving checkpoint...
# Done and done!
Epoch #16
# Time passed: 6 s
# Epoch losses | emotion = 1.1506 | speaker = 2.2658 | gender = 0.8107 | total = 1.6859 |
# Train accuracies | emotion = 0.4255144176931691 | speaker = 0.1252449608

## Saving best model_alex
# Done and done!
Epoch #28
# Time passed: 6 s
# Epoch losses | emotion = 1.0588 | speaker = 2.1875 | gender = 0.7820 | total = 1.6221 |
# Train accuracies | emotion = 0.4755439129739242 | speaker = 0.1392377219644857 | gender = 0.4545672692369221 |
# Validation process on validation set
# Validation losses | emotion = 0.0381 | speaker = 0.0708 | gender = 0.0241 | total = 0.0530 |
# Validation accuracies | emotion = 0.5677491601343785 | speaker = 0.21948488241881298 | gender = 0.4400895856662934 |
# Saving checkpoint...
# Done and done!
Epoch #29
# Time passed: 6 s
# Epoch losses | emotion = 1.0781 | speaker = 2.1926 | gender = 0.7861 | total = 1.6273 |
# Train accuracies | emotion = 0.4779318067729853 | speaker = 0.1406437039039271 | gender = 0.45398115611846934 |
# Validation process on validation set
# Validation losses | emotion = 0.0314 | speaker = 0.0677 | gender = 0.0249 | total = 0.0499 |
# Validation accuracies | emotion = 0.5879059350503919 | speaker 

# Done and done!
Epoch #42
# Time passed: 6 s
# Epoch losses | emotion = 1.0420 | speaker = 2.0441 | gender = 0.7465 | total = 1.5228 |
# Train accuracies | emotion = 0.5033994560870261 | speaker = 0.16470031461632806 | gender = 0.45106782914733645 |
# Validation process on validation set
# Validation losses | emotion = 0.0310 | speaker = 0.0617 | gender = 0.0241 | total = 0.0458 |
# Validation accuracies | emotion = 0.5823068309070548 | speaker = 0.2989921612541993 | gender = 0.41993281075027994 |
# Saving checkpoint...
# Done and done!
Epoch #43
# Time passed: 6 s
# Epoch losses | emotion = 1.0320 | speaker = 2.0382 | gender = 0.7444 | total = 1.5175 |
# Train accuracies | emotion = 0.5049089820047397 | speaker = 0.16666449647126227 | gender = 0.45093622229745567 |
# Validation process on validation set
# Validation losses | emotion = 0.0357 | speaker = 0.0593 | gender = 0.0235 | total = 0.0451 |
# Validation accuracies | emotion = 0.5823068309070548 | speaker = 0.2889137737961926 | 

# Done and done!
Epoch #56
# Time passed: 6 s
# Epoch losses | emotion = 1.0240 | speaker = 1.9164 | gender = 0.7218 | total = 1.4372 |
# Train accuracies | emotion = 0.519771836506159 | speaker = 0.1914543673012318 | gender = 0.4494830827067669 |
# Validation process on validation set
# Validation losses | emotion = 0.0341 | speaker = 0.0576 | gender = 0.0230 | total = 0.0436 |
# Validation accuracies | emotion = 0.5856662933930571 | speaker = 0.3169092945128779 | gender = 0.3650615901455767 |
# Saving checkpoint...
# Done and done!
Epoch #57
# Time passed: 6 s
# Epoch losses | emotion = 1.0171 | speaker = 1.8984 | gender = 0.7188 | total = 1.4243 |
# Train accuracies | emotion = 0.5208247382173238 | speaker = 0.19315435846054105 | gender = 0.4495884167305161 |
# Validation process on validation set
# Validation losses | emotion = 0.0371 | speaker = 0.0575 | gender = 0.0236 | total = 0.0443 |
# Validation accuracies | emotion = 0.568868980963046 | speaker = 0.3236282194848824 | gender

...

# Validation losses | emotion = 0.0272 | speaker = 0.0242 | gender = 0.0121 | total = 0.0232 |
# Validation accuracies | emotion = 0.6606942889137738 | speaker = 0.6237402015677491 | gender = 0.8611422172452408 |
# Saving checkpoint...
# Done and done!
Epoch #238
# Time passed: 6 s
# Epoch losses | emotion = 0.8005 | speaker = 0.8693 | gender = 0.3988 | total = 0.7631 |
# Train accuracies | emotion = 0.6047961737886643 | speaker = 0.42359222524396095 | gender = 0.6766235049450912 |
# Validation process on validation set
# Validation losses | emotion = 0.0357 | speaker = 0.0326 | gender = 0.0126 | total = 0.0309 |
# Validation accuracies | emotion = 0.6618141097424413 | speaker = 0.6304591265397537 | gender = 0.8689809630459127 |
# Saving checkpoint...
# Done and done!
Epoch #239
# Time passed: 6 s
# Epoch losses | emotion = 0.7998 | speaker = 0.8794 | gender = 0.3988 | total = 0.7673 |
# Train accuracies | emotion = 0.6050827683470227 | speaker = 0.4244987747567084 | gender = 0.6773592

# Validation losses | emotion = 0.0187 | speaker = 0.0307 | gender = 0.0102 | total = 0.0235 |
# Validation accuracies | emotion = 0.6640537513997761 | speaker = 0.6427771556550952 | gender = 0.8723404255319149 |
# Saving checkpoint...
# Done and done!
Epoch #252
# Time passed: 6 s
# Epoch losses | emotion = 0.7857 | speaker = 0.8554 | gender = 0.3809 | total = 0.7491 |
# Train accuracies | emotion = 0.6091991947954993 | speaker = 0.43599579622815904 | gender = 0.686475719440445 |
# Validation process on validation set
# Validation losses | emotion = 0.0293 | speaker = 0.0221 | gender = 0.0099 | total = 0.0236 |
# Validation accuracies | emotion = 0.6618141097424413 | speaker = 0.6338185890257558 | gender = 0.8712206047032475 |
# Saving checkpoint...
# Done and done!
Epoch #253
# Time passed: 6 s
# Epoch losses | emotion = 0.7754 | speaker = 0.8505 | gender = 0.3876 | total = 0.7442 |
# Train accuracies | emotion = 0.6095122804066765 | speaker = 0.4368640148011101 | gender = 0.68711630

# Validation losses | emotion = 0.0301 | speaker = 0.0292 | gender = 0.0122 | total = 0.0267 |
# Validation accuracies | emotion = 0.6662933930571109 | speaker = 0.631578947368421 | gender = 0.8768197088465846 |
# Saving checkpoint...
# Done and done!
Epoch #266
# Time passed: 6 s
# Epoch losses | emotion = 0.7636 | speaker = 0.7937 | gender = 0.3694 | total = 0.7111 |
# Train accuracies | emotion = 0.6134134327981208 | speaker = 0.44825880490700437 | gender = 0.6956171223130615 |
# Validation process on validation set
# Validation losses | emotion = 0.0419 | speaker = 0.0276 | gender = 0.0090 | total = 0.0331 |
# Validation accuracies | emotion = 0.6696528555431132 | speaker = 0.6304591265397537 | gender = 0.8790593505039194 |
# Saving checkpoint...
# Done and done!
Epoch #267
# Time passed: 6 s
# Epoch losses | emotion = 0.7675 | speaker = 0.8054 | gender = 0.3703 | total = 0.7176 |
# Train accuracies | emotion = 0.6136838330586207 | speaker = 0.4491068694926415 | gender = 0.69625069

# Validation losses | emotion = 0.0227 | speaker = 0.0289 | gender = 0.0130 | total = 0.0235 |
# Validation accuracies | emotion = 0.6651735722284434 | speaker = 0.6438969764837627 | gender = 0.8712206047032475 |
# Saving checkpoint...
# Done and done!
Epoch #280
# Time passed: 6 s
# Epoch losses | emotion = 0.7448 | speaker = 0.7730 | gender = 0.3568 | total = 0.6941 |
# Train accuracies | emotion = 0.617473204287314 | speaker = 0.459805431131019 | gender = 0.7041263397856343 |
# Validation process on validation set
# Validation losses | emotion = 0.0278 | speaker = 0.0378 | gender = 0.0078 | total = 0.0308 |
# Validation accuracies | emotion = 0.6562150055991042 | speaker = 0.6573348264277715 | gender = 0.8779395296752519 |
# Saving checkpoint...
# Done and done!
Epoch #281
# Time passed: 6 s
# Epoch losses | emotion = 0.7495 | speaker = 0.7730 | gender = 0.3567 | total = 0.6970 |
# Train accuracies | emotion = 0.6177744656940298 | speaker = 0.4606249476952015 | gender = 0.7047120147

# Validation losses | emotion = 0.0241 | speaker = 0.0318 | gender = 0.0098 | total = 0.0257 |
# Validation accuracies | emotion = 0.6774916013437849 | speaker = 0.6528555431131019 | gender = 0.8902575587905935 |
# Saving checkpoint...
# Done and done!
Epoch #294
# Time passed: 6 s
# Epoch losses | emotion = 0.7270 | speaker = 0.7303 | gender = 0.3437 | total = 0.6665 |
# Train accuracies | emotion = 0.6214996076818186 | speaker = 0.4710417761729552 | gender = 0.7121174897730649 |
# Validation process on validation set
# Validation losses | emotion = 0.0267 | speaker = 0.0179 | gender = 0.0113 | total = 0.0208 |
# Validation accuracies | emotion = 0.6730123180291153 | speaker = 0.6550951847704367 | gender = 0.8947368421052632 |
# Saving checkpoint...
# Done and done!
Epoch #295
# Time passed: 6 s
# Epoch losses | emotion = 0.7281 | speaker = 0.7473 | gender = 0.3460 | total = 0.6759 |
# Train accuracies | emotion = 0.6217719741112608 | speaker = 0.47179285212671057 | gender = 0.7126511

In [5]:
ts.create_results_file()

In [6]:
results = pd.read_csv(os.path.join(ts.path_to_results, f'{ts.name}_results.csv'), delimiter=';')
results_test = results.loc[results['subset']=='test']
results_test_accuracy = results_test.loc[results_test['metric']=='accuracy']
results_test_accuracy.drop('epoch', axis=1, inplace=True)
results_test_accuracy.groupby('task').max()

/home/aggr/anaconda3/envs/ryabinov/lib/python3.8/site-packages/pandas/core/frame.py:3990: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,result,subset,metric
task,,,
emotion,0.685330,test,accuracy
gender,0.894737,test,accuracy
speaker,0.659574,test,accuracy


#### Теперь попробуем другую стратегию взвешивания, из статьи

In [2]:
train_ds = ds.IemocapDataset(  # Без препроцессинга, тренировочный
    PATH_TO_PICKLE, IEMOCAP_PATH_TO_WAVS, IEMOCAP_PATH_TO_EGEMAPS, IEMOCAP_PATH_FOR_PARSER, 
    base_name='IEMOCAP-4', label_type='four', mode='train', preprocessing=False, 
    augmentation=True, padding='repeat', spectrogram_shape=224, spectrogram_type='melspec', tasks=('emotion', 'speaker', 'gender') 
)
test_ds = ds.IemocapDataset(  # Без препроцессинга, тестовый
    PATH_TO_PICKLE, IEMOCAP_PATH_TO_WAVS, IEMOCAP_PATH_TO_EGEMAPS, IEMOCAP_PATH_FOR_PARSER, 
    base_name='IEMOCAP-4', label_type='four', mode='test', preprocessing=False, 
    augmentation=True, padding='repeat', spectrogram_shape=224, spectrogram_type='melspec', tasks=('emotion', 'speaker', 'gender') 
)

============= INITIALIZING DATASET IEMOCAP-4_four_prep-false_224_train ===============
=========================== SUCCESS! ====================================
============= INITIALIZING DATASET IEMOCAP-4_four_prep-false_224_test ===============
=========================== SUCCESS! ====================================


In [3]:
model = md_multi.AlexNetMultiTask(4, 10, 2)
awl = AutomaticWeightedLoss(num=3)
optimizer = torch.optim.Adam([
                {'params': model.parameters(), 'lr': 3e-6}, 
                {'params': awl.parameters(), 'weight_decay': 0}
            ])
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 
# device = torch.device("cpu") 
criterion = nn.CrossEntropyLoss()
ts = TrainingSession(name='UncertaintyWeighting',
                      model=model,
                      train_dataset=train_ds,
                      test_dataset=test_ds,
                      criterion=criterion,
                      optimizer=optimizer,
                      num_epochs=400,
                      batch_size=32,
                      device=device,
                     path_to_weights=WEIGHTS_FOLDER,
                     path_to_results=RESULTS_FOLDER,
                    loss_weighter=awl  # Из файла multitask_training_session.py
                    )

INITIALIZING TRAINING SESSION...
Loaders ready
TRAINING SESSION UncertaintyWeighting__IEMOCAP-4_four_prep-false_224_train INITIALIZED
Trying to load checkpoint from file
File not found, starting from scratch...


#### Оверфитим батч...

In [4]:
ts.overfit_one_batch(num_epochs=15, batch_size=32)  # 15 эпох на одном батче размером 32, в одной эпохе 50 одинаковых батчей

TRAIN SIZE 2858
TEST SIZE 714
Epoch 0
Emotion loss | Speaker loss | Gender loss | Total loss
0: 1.3924750089645386 | 2.303546667098999 | 0.6955535411834717 | 4.275229454040527
1: 1.3909106254577637 | 2.302889108657837 | 0.6981048583984375 | 4.273404598236084
2: 1.3898227214813232 | 2.3014354705810547 | 0.6961358785629272 | 4.269175052642822
3: 1.3905456066131592 | 2.2993619441986084 | 0.6962890625 | 4.266619682312012
4: 1.3923399448394775 | 2.298981189727783 | 0.6951851844787598 | 4.2648186683654785
5: 1.3906126022338867 | 2.302354335784912 | 0.6942808032035828 | 4.263242721557617
6: 1.3915672302246094 | 2.2999987602233887 | 0.6944597959518433 | 4.260717868804932
7: 1.3933054208755493 | 2.2974960803985596 | 0.6973386406898499 | 4.259893417358398
8: 1.3943363428115845 | 2.2984397411346436 | 0.6951867938041687 | 4.2578840255737305
9: 1.3900257349014282 | 2.2982075214385986 | 0.6925908327102661 | 4.252452850341797
10: 1.3945631980895996 | 2.298682689666748 | 0.6921578049659729 | 4.2528271

44: 1.409702181816101 | 2.087980031967163 | 0.7092665433883667 | 4.076523780822754
45: 1.344657063484192 | 2.1380655765533447 | 0.6970045566558838 | 4.061939239501953
46: 1.4161076545715332 | 1.9928895235061646 | 0.7001873254776001 | 4.03229284286499
47: 1.4362465143203735 | 2.1187734603881836 | 0.7049124240875244 | 4.096724987030029
48: 1.4071083068847656 | 2.127992630004883 | 0.6954562664031982 | 4.081886291503906
49: 1.3511825799942017 | 2.179415464401245 | 0.7107644081115723 | 4.088132381439209
# Time passed: 1 s
# Epoch losses | emotion = 1.3857 | speaker = 2.1234 | gender = 0.6933 |
# Train accuracies | emotion = 0.315625 | speaker = 0.190625 | gender = 0.545 |
Epoch 2
Emotion loss | Speaker loss | Gender loss | Total loss
0: 1.404913067817688 | 2.058640956878662 | 0.6627728343009949 | 4.030973434448242
1: 1.4115355014801025 | 2.0302462577819824 | 0.7167372107505798 | 4.053386211395264
2: 1.359991431236267 | 2.054213047027588 | 0.6843921542167664 | 4.021616458892822
3: 1.38274776

39: 1.371170997619629 | 2.1076221466064453 | 0.7033069729804993 | 4.028951644897461
40: 1.3800320625305176 | 2.065272569656372 | 0.7107149362564087 | 4.0208611488342285
41: 1.3423773050308228 | 2.0763113498687744 | 0.6715126633644104 | 3.984592914581299
42: 1.377423882484436 | 2.0588808059692383 | 0.6776229739189148 | 3.995917797088623
43: 1.406693935394287 | 2.0511839389801025 | 0.6913997530937195 | 4.013472080230713
44: 1.3597309589385986 | 2.0798981189727783 | 0.6814631223678589 | 3.998718500137329
45: 1.370368480682373 | 2.0572140216827393 | 0.6828320622444153 | 3.9950900077819824
46: 1.386906623840332 | 2.083690881729126 | 0.7093802690505981 | 4.028566360473633
47: 1.3489534854888916 | 2.0934488773345947 | 0.6598154902458191 | 3.9850778579711914
48: 1.33889901638031 | 2.0859129428863525 | 0.6974166035652161 | 4.001817226409912
49: 1.3928303718566895 | 1.9922122955322266 | 0.6569764614105225 | 3.962374448776245
# Time passed: 1 s
# Epoch losses | emotion = 1.3761 | speaker = 2.0892

30: 1.254611849784851 | 1.8786989450454712 | 0.5373443365097046 | 3.775320053100586
31: 1.2261605262756348 | 1.8471615314483643 | 0.5425765514373779 | 3.755556583404541
32: 1.2409497499465942 | 1.862103819847107 | 0.5787292718887329 | 3.791313648223877
33: 1.1120550632476807 | 1.9468549489974976 | 0.5901045203208923 | 3.7756569385528564
34: 1.0957510471343994 | 1.7304890155792236 | 0.5202770829200745 | 3.643855333328247
35: 1.141526699066162 | 1.882583737373352 | 0.5487494468688965 | 3.736482620239258
36: 1.1721574068069458 | 1.7970759868621826 | 0.540433943271637 | 3.712790012359619
37: 1.0616271495819092 | 1.9052687883377075 | 0.4821186363697052 | 3.665353775024414
38: 1.0410091876983643 | 1.8172188997268677 | 0.5276131629943848 | 3.6559810638427734
39: 1.1934216022491455 | 1.8741397857666016 | 0.4959527254104614 | 3.7181520462036133
40: 1.1575263738632202 | 1.688137173652649 | 0.500244140625 | 3.6392998695373535
41: 1.1944948434829712 | 1.585698127746582 | 0.471813440322876 | 3.5983

21: 0.6774816513061523 | 1.281148910522461 | 0.28950047492980957 | 3.0720365047454834
22: 0.6056893467903137 | 1.2138760089874268 | 0.2751118242740631 | 2.998629570007324
23: 0.7346875071525574 | 1.2712656259536743 | 0.3695654273033142 | 3.1646974086761475
24: 0.8382973074913025 | 1.1245355606079102 | 0.29701027274131775 | 3.099118232727051
25: 0.8646655082702637 | 1.2729371786117554 | 0.27554425597190857 | 3.146355152130127
26: 0.8593478798866272 | 1.1950163841247559 | 0.3134414851665497 | 3.148169755935669
27: 0.885701060295105 | 1.2699030637741089 | 0.19740284979343414 | 3.0850164890289307
28: 0.7344315648078918 | 1.0808498859405518 | 0.3842920958995819 | 3.1055893898010254
29: 0.8888488411903381 | 1.2733830213546753 | 0.18939948081970215 | 3.0786595344543457
30: 0.6961773037910461 | 1.0709182024002075 | 0.37824878096580505 | 3.0760912895202637
31: 0.792913556098938 | 1.2993621826171875 | 0.23884136974811554 | 3.0818729400634766
32: 0.7023917436599731 | 1.277733325958252 | 0.3435120

16: 0.3288455307483673 | 0.7343376874923706 | 0.11483950167894363 | 2.3459529876708984
17: 0.4032648205757141 | 0.7114564180374146 | 0.09646207839250565 | 2.359863758087158
18: 0.2841699421405792 | 0.6292693614959717 | 0.12783411145210266 | 2.2878081798553467
19: 0.3011936843395233 | 0.9068723917007446 | 0.10135472565889359 | 2.3717474937438965
20: 0.45809879899024963 | 0.7639552354812622 | 0.16241538524627686 | 2.507716655731201
21: 0.4102514088153839 | 0.6723136901855469 | 0.13973745703697205 | 2.4039831161499023
22: 0.38418495655059814 | 0.7010524868965149 | 0.06720959395170212 | 2.2912487983703613
23: 0.37809962034225464 | 0.7566217184066772 | 0.1161491870880127 | 2.378786087036133
24: 0.2694767713546753 | 0.7919483184814453 | 0.15498119592666626 | 2.372382164001465
25: 0.3591393530368805 | 0.7675029039382935 | 0.054313357919454575 | 2.2758896350860596
26: 0.35497260093688965 | 0.7515034675598145 | 0.06651241332292557 | 2.2824392318725586
27: 0.5259528160095215 | 0.8264783620834351

3: 0.16402015089988708 | 0.3000558912754059 | 0.01691855862736702 | 1.652840495109558
4: 0.21104282140731812 | 0.4876621663570404 | 0.019210699945688248 | 1.7930939197540283
5: 0.1327296495437622 | 0.4373229444026947 | 0.021819986402988434 | 1.6938247680664062
6: 0.13909576833248138 | 0.4048216640949249 | 0.00846908614039421 | 1.6478074789047241
7: 0.3269844949245453 | 0.5204390287399292 | 0.021202927455306053 | 1.926845669746399
8: 0.10143755376338959 | 0.3776843845844269 | 0.030243350192904472 | 1.6472876071929932
9: 0.10826466977596283 | 0.4499504268169403 | 0.02599530667066574 | 1.6756069660186768
10: 0.10300401598215103 | 0.3752394914627075 | 0.01575443148612976 | 1.6043064594268799
11: 0.0911179855465889 | 0.42669418454170227 | 0.005264446139335632 | 1.5859298706054688
12: 0.17902909219264984 | 0.42341673374176025 | 0.018702303990721703 | 1.7103043794631958
13: 0.3527459502220154 | 0.497947096824646 | 0.0033142429310828447 | 1.8866835832595825
14: 0.0719681829214096 | 0.357733368

46: 0.15626540780067444 | 0.21436730027198792 | 0.0019590803422033787 | 1.3758198022842407
47: 0.06279841810464859 | 0.3446325659751892 | 0.013353249989449978 | 1.359083890914917
48: 0.03659636154770851 | 0.12020225077867508 | 0.013154435902833939 | 1.1808243989944458
49: 0.07657468318939209 | 0.12032442539930344 | 0.005446807947009802 | 1.2076976299285889
# Time passed: 1 s
# Epoch losses | emotion = 0.1076 | speaker = 0.2753 | gender = 0.0185 |
# Train accuracies | emotion = 0.965625 | speaker = 0.91625 | gender = 0.996875 |
Epoch 13
Emotion loss | Speaker loss | Gender loss | Total loss
0: 0.1082598939538002 | 0.35619649291038513 | 0.025601506233215332 | 1.4840404987335205
1: 0.07462969422340393 | 0.3053629398345947 | 0.018178720027208328 | 1.36844801902771
2: 0.050665151327848434 | 0.19446709752082825 | 0.0025146501138806343 | 1.1950682401657104
3: 0.03754182532429695 | 0.3144483268260956 | 0.0035910806618630886 | 1.2510298490524292
4: 0.10805144906044006 | 0.3263171315193176 | 0.0

38: 0.012624692171812057 | 0.08667699247598648 | 0.006382917519658804 | 0.9004037380218506
39: 0.00679046381264925 | 0.11776979267597198 | 0.008895196951925755 | 0.9264020323753357
40: 0.14226342737674713 | 0.12227463722229004 | 0.0013838043669238687 | 1.1942745447158813
41: 0.03718734532594681 | 0.2804044485092163 | 0.0028575381729751825 | 1.0818051099777222
42: 0.1561080813407898 | 0.03754473477602005 | 0.004306243732571602 | 1.1786249876022339
43: 0.008550664409995079 | 0.1332412213087082 | 0.00046823156299069524 | 0.8781353831291199
44: 0.038737326860427856 | 0.14449797570705414 | 0.00851894449442625 | 1.0132229328155518
45: 0.12162071466445923 | 0.08147227764129639 | 0.0011202107416465878 | 1.1071234941482544
46: 0.031055085361003876 | 0.1109246015548706 | 0.01196221262216568 | 0.9899766445159912
47: 0.010620979592204094 | 0.12760519981384277 | 0.006083048414438963 | 0.911754846572876
48: 0.058488719165325165 | 0.1404389590024948 | 0.005231287330389023 | 1.0295623540878296
49: 0.0

### Тоже работает, запускаем!

In [4]:
ts.execute()

=============TRAINING SESSION STARTED AT EPOCH 1=====================
Epoch #1
# Time passed: 6 s
# Epoch losses | emotion = 1.3805 | speaker = 2.2993 | gender = 0.6931 | total = 4.1854 |
# Train accuracies | emotion = 0.3491041433370661 | speaker = 0.11674132138857783 | gender = 0.5106382978723404 |
# Validation process on validation set
# Validation losses | emotion = 0.0440 | speaker = 0.0751 | gender = 0.0226 | total = 0.1340 |
# Validation accuracies | emotion = 0.3807390817469205 | speaker = 0.11534154535274356 | gender = 0.5005599104143337 |
# Saving checkpoint...
## Saving best model_alex
# Done and done!
Epoch #2
# Time passed: 6 s
# Epoch losses | emotion = 1.3542 | speaker = 2.2968 | gender = 0.6929 | total = 4.0916 |
# Train accuracies | emotion = 0.3649216125419933 | speaker = 0.11884098544232923 | gender = 0.511478163493841 |
# Validation process on validation set
# Validation losses | emotion = 0.0415 | speaker = 0.0744 | gender = 0.0225 | total = 0.1313 |
# Validation a

# Validation losses | emotion = 0.0380 | speaker = 0.0753 | gender = 0.0182 | total = 0.1261 |
# Validation accuracies | emotion = 0.49496080627099664 | speaker = 0.135498320268757 | gender = 0.6517357222844344 |
# Saving checkpoint...
## Saving best model_alex
# Done and done!
Epoch #15
# Time passed: 6 s
# Epoch losses | emotion = 1.2066 | speaker = 2.2846 | gender = 0.6383 | total = 3.9394 |
# Train accuracies | emotion = 0.3932624113475177 | speaker = 0.11879432624113476 | gender = 0.5470324748040314 |
# Validation process on validation set
# Validation losses | emotion = 0.0375 | speaker = 0.0726 | gender = 0.0185 | total = 0.1253 |
# Validation accuracies | emotion = 0.49832026875699886 | speaker = 0.1455767077267637 | gender = 0.6730123180291153 |
# Saving checkpoint...
## Saving best model_alex
# Done and done!
Epoch #16
# Time passed: 6 s
# Epoch losses | emotion = 1.1832 | speaker = 2.2847 | gender = 0.6268 | total = 3.9213 |
# Train accuracies | emotion = 0.3983762597984323 

# Validation losses | emotion = 0.0383 | speaker = 0.0733 | gender = 0.0156 | total = 0.1238 |
# Validation accuracies | emotion = 0.48712206047032475 | speaker = 0.14109742441209405 | gender = 0.7166853303471444 |
# Saving checkpoint...
# Done and done!
Epoch #29
# Time passed: 6 s
# Epoch losses | emotion = 1.1418 | speaker = 2.2760 | gender = 0.5694 | total = 3.8601 |
# Train accuracies | emotion = 0.4367204695524578 | speaker = 0.12297756496891532 | gender = 0.6126288759315751 |
# Validation process on validation set
# Validation losses | emotion = 0.0332 | speaker = 0.0739 | gender = 0.0121 | total = 0.1190 |
# Validation accuracies | emotion = 0.48600223964165734 | speaker = 0.16349384098544234 | gender = 0.7189249720044792 |
# Saving checkpoint...
# Done and done!
Epoch #30
# Time passed: 6 s
# Epoch losses | emotion = 1.1475 | speaker = 2.2693 | gender = 0.5674 | total = 3.8594 |
# Train accuracies | emotion = 0.4383631952220978 | speaker = 0.12341358715938783 | gender = 0.6158

# Validation losses | emotion = 0.0409 | speaker = 0.0728 | gender = 0.0089 | total = 0.1188 |
# Validation accuracies | emotion = 0.5139977603583427 | speaker = 0.16685330347144456 | gender = 0.8533034714445689 |
# Saving checkpoint...
## Saving best model_alex
# Done and done!
Epoch #43
# Time passed: 6 s
# Epoch losses | emotion = 1.1339 | speaker = 2.2726 | gender = 0.4016 | total = 3.7214 |
# Train accuracies | emotion = 0.4533972238860387 | speaker = 0.12784187088205423 | gender = 0.6633440975025391 |
# Validation process on validation set
# Validation losses | emotion = 0.0305 | speaker = 0.0705 | gender = 0.0072 | total = 0.1118 |
# Validation accuracies | emotion = 0.5251959686450168 | speaker = 0.18029115341545351 | gender = 0.8611422172452408 |
# Saving checkpoint...
## Saving best model_alex
# Done and done!
Epoch #44
# Time passed: 6 s
# Epoch losses | emotion = 1.1220 | speaker = 2.2709 | gender = 0.3941 | total = 3.7087 |
# Train accuracies | emotion = 0.4546727069123485

# Done and done!
Epoch #56
# Time passed: 6 s
# Epoch losses | emotion = 1.0675 | speaker = 2.2557 | gender = 0.3228 | total = 3.6107 |
# Train accuracies | emotion = 0.47158954567269235 | speaker = 0.13189889617661174 | gender = 0.706546952487602 |
# Validation process on validation set
# Validation losses | emotion = 0.0319 | speaker = 0.0763 | gender = 0.0098 | total = 0.1162 |
# Validation accuracies | emotion = 0.561030235162374 | speaker = 0.17581187010078386 | gender = 0.8868980963045913 |
# Saving checkpoint...
# Done and done!
Epoch #57
# Time passed: 6 s
# Epoch losses | emotion = 1.0683 | speaker = 2.2579 | gender = 0.3208 | total = 3.6096 |
# Train accuracies | emotion = 0.4729916897506925 | speaker = 0.13213885778275475 | gender = 0.709386848981356 |
# Validation process on validation set
# Validation losses | emotion = 0.0341 | speaker = 0.0692 | gender = 0.0056 | total = 0.1109 |
# Validation accuracies | emotion = 0.5789473684210527 | speaker = 0.1623740201567749 | gend

...

# Validation losses | emotion = 0.0402 | speaker = 0.0573 | gender = 0.0005 | total = 0.0984 |
# Validation accuracies | emotion = 0.6226203807390818 | speaker = 0.3337066069428891 | gender = 0.9675251959686451 |
# Saving checkpoint...
# Done and done!
Epoch #336
# Time passed: 6 s
# Epoch losses | emotion = 0.9339 | speaker = 1.9403 | gender = 0.0512 | total = 3.0291 |
# Train accuracies | emotion = 0.5745430731083027 | speaker = 0.20798172292433212 | gender = 0.9113383525302619 |
# Validation process on validation set
# Validation losses | emotion = 0.0260 | speaker = 0.0651 | gender = 0.0013 | total = 0.0957 |
# Validation accuracies | emotion = 0.620380739081747 | speaker = 0.3325867861142217 | gender = 0.9675251959686451 |
# Saving checkpoint...
# Done and done!
Epoch #337
# Time passed: 6 s
# Epoch losses | emotion = 0.9369 | speaker = 1.9468 | gender = 0.0507 | total = 3.0312 |
# Train accuracies | emotion = 0.5746699519174855 | speaker = 0.2082476299341067 | gender = 0.91155243

# Validation losses | emotion = 0.0298 | speaker = 0.0619 | gender = 0.0079 | total = 0.1149 |
# Validation accuracies | emotion = 0.6259798432250839 | speaker = 0.3202687569988802 | gender = 0.9720044792833147 |
# Saving checkpoint...
# Done and done!
Epoch #350
# Time passed: 6 s
# Epoch losses | emotion = 0.9244 | speaker = 1.9347 | gender = 0.0439 | total = 3.0015 |
# Train accuracies | emotion = 0.5764237721964486 | speaker = 0.21152375619900815 | gender = 0.9141513357862742 |
# Validation process on validation set
# Validation losses | emotion = 0.0246 | speaker = 0.0501 | gender = 0.0187 | total = 0.1420 |
# Validation accuracies | emotion = 0.6338185890257558 | speaker = 0.3247480403135498 | gender = 0.9686450167973124 |
# Saving checkpoint...
## Saving best model_alex
# Done and done!
Epoch #351
# Time passed: 6 s
# Epoch losses | emotion = 0.9111 | speaker = 1.9334 | gender = 0.0503 | total = 3.0123 |
# Train accuracies | emotion = 0.576567350363543 | speaker = 0.211801667288

# Validation losses | emotion = 0.0224 | speaker = 0.0571 | gender = 0.0104 | total = 0.1188 |
# Validation accuracies | emotion = 0.6326987681970885 | speaker = 0.341545352743561 | gender = 0.9641657334826428 |
# Saving checkpoint...
# Done and done!
Epoch #364
# Time passed: 6 s
# Epoch losses | emotion = 0.9295 | speaker = 1.9311 | gender = 0.0535 | total = 3.0309 |
# Train accuracies | emotion = 0.5783205456357752 | speaker = 0.215087124521615 | gender = 0.916768701623125 |
# Validation process on validation set
# Validation losses | emotion = 0.0254 | speaker = 0.0581 | gender = 0.0023 | total = 0.0961 |
# Validation accuracies | emotion = 0.6248600223964166 | speaker = 0.335946248600224 | gender = 0.9708846584546472 |
# Saving checkpoint...
# Done and done!
Epoch #365
# Time passed: 6 s
# Epoch losses | emotion = 0.9181 | speaker = 1.9186 | gender = 0.0463 | total = 2.9994 |
# Train accuracies | emotion = 0.5784595560600715 | speaker = 0.21531700133458098 | gender = 0.91694764454

# Validation losses | emotion = 0.0312 | speaker = 0.0587 | gender = 0.0001 | total = 0.0933 |
# Validation accuracies | emotion = 0.620380739081747 | speaker = 0.3460246360582307 | gender = 0.9664053751399776 |
# Saving checkpoint...
# Done and done!
Epoch #378
# Time passed: 6 s
# Epoch losses | emotion = 0.9116 | speaker = 1.8963 | gender = 0.0443 | total = 2.9835 |
# Train accuracies | emotion = 0.5802167949424388 | speaker = 0.2185220735052762 | gender = 0.9192210727765039 |
# Validation process on validation set
# Validation losses | emotion = 0.0322 | speaker = 0.0683 | gender = 0.0100 | total = 0.1260 |
# Validation accuracies | emotion = 0.6215005599104143 | speaker = 0.3236282194848824 | gender = 0.961926091825308 |
# Saving checkpoint...
# Done and done!
Epoch #379
# Time passed: 6 s
# Epoch losses | emotion = 0.9204 | speaker = 1.9073 | gender = 0.0437 | total = 2.9892 |
# Train accuracies | emotion = 0.5803227684098249 | speaker = 0.21877797705401436 | gender = 0.919397276

# Validation losses | emotion = 0.0291 | speaker = 0.0631 | gender = 0.0167 | total = 0.1446 |
# Validation accuracies | emotion = 0.6282194848824189 | speaker = 0.3404255319148936 | gender = 0.9697648376259799 |
# Saving checkpoint...
# Done and done!
Epoch #392
# Time passed: 6 s
# Epoch losses | emotion = 0.9116 | speaker = 1.8906 | gender = 0.0474 | total = 2.9904 |
# Train accuracies | emotion = 0.5819747411842676 | speaker = 0.22187378590854034 | gender = 0.9215404106771488 |
# Validation process on validation set
# Validation losses | emotion = 0.0301 | speaker = 0.0653 | gender = 0.0119 | total = 0.1300 |
# Validation accuracies | emotion = 0.6091825307950728 | speaker = 0.3449048152295633 | gender = 0.9697648376259799 |
# Saving checkpoint...
# Done and done!
Epoch #393
# Time passed: 6 s
# Epoch losses | emotion = 0.9111 | speaker = 1.8784 | gender = 0.0411 | total = 2.9667 |
# Train accuracies | emotion = 0.5821038099552927 | speaker = 0.22213199068810568 | gender = 0.921703

In [5]:
ts.create_results_file()

In [6]:
results = pd.read_csv(os.path.join(ts.path_to_results, f'{ts.name}_results.csv'), delimiter=';')
results_test = results.loc[results['subset']=='test']
results_test_accuracy = results_test.loc[results_test['metric']=='accuracy']
results_test_accuracy.drop('epoch', axis=1, inplace=True)
results_test_accuracy.groupby('task').max()

/home/aggr/anaconda3/envs/ryabinov/lib/python3.8/site-packages/pandas/core/frame.py:3990: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,result,subset,metric
task,,,
emotion,0.636058,test,accuracy
gender,0.974244,test,accuracy
speaker,0.351624,test,accuracy


## Обучаем модель на RAMAS

In [2]:
ramas_224_train = RamasDataset(RAMAS_PATH_TO_WAVS, 'Ramas224BinaryMulti',
                 spectrogram_shape=224,
                 augmentation=True, padding='repeat', mode='train',  tasks='multi', type='binary')
ramas_224_test = RamasDataset(RAMAS_PATH_TO_WAVS, 'Ramas224BinaryMulti',
                 spectrogram_shape=224,
                 augmentation=False, padding='repeat', mode='test',  tasks='multi', type='binary')

============= INITIALIZING DATASET Ramas224BinaryMulti_224_train ===============
============================ SUCCESS! =========================
============= INITIALIZING DATASET Ramas224BinaryMulti_224_test ===============
============================ SUCCESS! =========================


In [3]:
train_ds = ramas_224_train
test_ds = ramas_224_test
model = md_multi.AlexNetMultiTask(num_emotions=2, num_speakers=12, num_genders=2)
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu") 
# device = torch.device("cpu") 
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model.parameters(), lr=1e-4)
ts = TrainingSession(name='AverageWeighting',
                      model=model,
                      train_dataset=train_ds,
                      test_dataset=test_ds,
                      criterion=criterion,
                      optimizer=optimizer,
                      num_epochs=300,
                      batch_size=32,
                      device=device,
                     path_to_weights=WEIGHTS_FOLDER,
                     path_to_results=RESULTS_FOLDER,
                    loss_weighter=averaged_sum  # Из файла multitask_training_session.py
                    )

INITIALIZING TRAINING SESSION...
Loaders ready
TRAINING SESSION AverageWeighting__Ramas224BinaryMulti_224_train INITIALIZED
Trying to load checkpoint from file
Found file
Loading file models/training_sessions/AverageWeighting__Ramas224BinaryMulti_224_train.pt
Updating model...
Updating optimizer...
Success!


In [12]:
ts.overfit_one_batch(num_epochs=15, batch_size=32)  # 15 эпох на одном батче размером 32, в одной эпохе 50 одинаковых батчей

TRAIN SIZE 3122
TEST SIZE 780
Epoch 0
Emotion loss | Speaker loss | Gender loss | Total loss
0: 0.7033777236938477 | 2.4895224571228027 | 0.6906255483627319 | 1.8461124897003174
1: 0.7069852352142334 | 2.4849765300750732 | 0.6903237700462341 | 1.8420807123184204
2: 0.708275318145752 | 2.4882090091705322 | 0.6895025968551636 | 1.8446413278579712
3: 0.7099183797836304 | 2.4878764152526855 | 0.69014972448349 | 1.8441157341003418
4: 0.7089220285415649 | 2.4895520210266113 | 0.6894952654838562 | 1.8456534147262573
5: 0.7135519981384277 | 2.484621286392212 | 0.692232608795166 | 1.8408581018447876
6: 0.7115105986595154 | 2.4838790893554688 | 0.6903555393218994 | 1.8407005071640015
7: 0.7156612873077393 | 2.48388409614563 | 0.6898545026779175 | 1.840322494506836
8: 0.7172453999519348 | 2.48327898979187 | 0.6914844512939453 | 1.8394783735275269
9: 0.7131936550140381 | 2.4818084239959717 | 0.6889252066612244 | 1.8390241861343384
10: 0.7196462750434875 | 2.4829225540161133 | 0.6905120611190796 | 

47: 0.8532489538192749 | 2.28834867477417 | 0.7775585055351257 | 1.6761696338653564
48: 0.8755112886428833 | 2.2912890911102295 | 0.8002316951751709 | 1.6780544519424438
49: 0.7916785478591919 | 2.293833017349243 | 0.7712051868438721 | 1.6810107231140137
# Time passed: 2 s
# Epoch losses | emotion = 0.8573 | speaker = 2.2658 | gender = 0.8364 |
# Train accuracies | emotion = 0.22875 | speaker = 0.22 | gender = 0.375625 |
Epoch 2
Emotion loss | Speaker loss | Gender loss | Total loss
0: 0.8337287306785583 | 2.1743040084838867 | 0.7482646107673645 | 1.5926858186721802
1: 0.8065701127052307 | 2.2714757919311523 | 0.8035231828689575 | 1.6631953716278076
2: 0.8523982167243958 | 2.260986089706421 | 0.8148949146270752 | 1.655354380607605
3: 0.7923014760017395 | 2.2695789337158203 | 0.8355008959770203 | 1.6618317365646362
4: 0.8132197260856628 | 2.318169116973877 | 0.8219221234321594 | 1.6975113153457642
5: 0.8472503423690796 | 2.1905879974365234 | 0.8192539215087891 | 1.6042358875274658
6: 0.

37: 0.801152765750885 | 2.097198247909546 | 0.8204030394554138 | 1.5363069772720337
38: 0.7638124823570251 | 2.134270668029785 | 0.7621790170669556 | 1.5625760555267334
39: 0.7828941345214844 | 2.128481388092041 | 0.7860372066497803 | 1.558173656463623
40: 0.7379764914512634 | 2.0941901206970215 | 0.7615175843238831 | 1.5332874059677124
41: 0.7750697135925293 | 2.0177652835845947 | 0.8298269510269165 | 1.4797743558883667
42: 0.8272234201431274 | 2.1066362857818604 | 0.7517662644386292 | 1.543121099472046
43: 0.8092553019523621 | 2.0578129291534424 | 0.8126876354217529 | 1.5082385540008545
44: 0.7438308596611023 | 2.038376808166504 | 0.816179096698761 | 1.4935616254806519
45: 0.7330865859985352 | 2.029494524002075 | 0.7661433219909668 | 1.4858739376068115
46: 0.7786849141120911 | 2.117400884628296 | 0.8121964931488037 | 1.5504212379455566
47: 0.7563484907150269 | 2.0884547233581543 | 0.754950225353241 | 1.528898000717163
48: 0.6710094809532166 | 2.1274499893188477 | 0.6619251370429993 |

31: 0.7283169031143188 | 1.7514433860778809 | 0.6898617148399353 | 1.2852979898452759
32: 0.7176724076271057 | 1.9026939868927002 | 0.7090683579444885 | 1.393052101135254
33: 0.6022966504096985 | 1.8272632360458374 | 0.7195663452148438 | 1.3398724794387817
34: 0.5988363027572632 | 1.8182305097579956 | 0.6497423648834229 | 1.3325681686401367
35: 0.7285444736480713 | 1.9166887998580933 | 0.6880607604980469 | 1.4033868312835693
36: 0.62347412109375 | 1.8197495937347412 | 0.6675536036491394 | 1.3327332735061646
37: 0.6794600486755371 | 1.879494309425354 | 0.7003551125526428 | 1.3759548664093018
38: 0.6816833019256592 | 1.7330822944641113 | 0.6513540744781494 | 1.269529104232788
39: 0.6982559561729431 | 1.7610281705856323 | 0.7388074994087219 | 1.2928392887115479
40: 0.7816998362541199 | 1.6341803073883057 | 0.693618893623352 | 1.2100685834884644
41: 0.6940910220146179 | 1.7127598524093628 | 0.6745427250862122 | 1.2560279369354248
42: 0.6271376609802246 | 1.7576872110366821 | 0.611123204231

21: 0.4338490664958954 | 1.3050284385681152 | 0.6259742379188538 | 0.9654592275619507
22: 0.640885055065155 | 1.319626808166504 | 0.5220370888710022 | 0.9766862392425537
23: 0.40969642996788025 | 1.308494210243225 | 0.6056689620018005 | 0.9668582677841187
24: 0.4515238106250763 | 1.1337075233459473 | 0.571504533290863 | 0.8419129848480225
25: 0.539728581905365 | 1.1896408796310425 | 0.5516014099121094 | 0.8815616965293884
26: 0.4970453977584839 | 1.4903968572616577 | 0.5129987597465515 | 1.0924091339111328
27: 0.5511756539344788 | 1.3318177461624146 | 0.6283659934997559 | 0.9844774603843689
28: 0.5373159050941467 | 1.2602421045303345 | 0.42611584067344666 | 0.9257172346115112
29: 0.47487789392471313 | 1.363356351852417 | 0.6055617332458496 | 1.0029293298721313
30: 0.5381676554679871 | 1.219904899597168 | 0.5088097453117371 | 0.8984496593475342
31: 0.544339120388031 | 1.2953845262527466 | 0.6263221502304077 | 0.959676206111908
32: 0.47281613945961 | 1.4198874235153198 | 0.49958702921867

13: 0.41685599088668823 | 0.8937358260154724 | 0.46775907278060913 | 0.6699078679084778
14: 0.3954375684261322 | 0.9149456024169922 | 0.446294367313385 | 0.6789378523826599
15: 0.34527966380119324 | 0.7720101475715637 | 0.44676318764686584 | 0.5849003195762634
16: 0.34037697315216064 | 0.8556047081947327 | 0.5536374449729919 | 0.6598180532455444
17: 0.3190423846244812 | 0.9348952174186707 | 0.49443501234054565 | 0.6979537010192871
18: 0.2872175872325897 | 0.9593287706375122 | 0.3604806661605835 | 0.7048742175102234
19: 0.3011600375175476 | 0.688187301158905 | 0.5200076103210449 | 0.5530223846435547
20: 0.2985219657421112 | 0.8216009736061096 | 0.46291661262512207 | 0.6180738210678101
21: 0.3729190528392792 | 0.8488060832023621 | 0.46347948908805847 | 0.6375211477279663
22: 0.31638583540916443 | 0.7027227878570557 | 0.5089746117591858 | 0.5581989288330078
23: 0.29949599504470825 | 0.8739093542098999 | 0.501303493976593 | 0.6596493721008301
24: 0.21179847419261932 | 0.723686158657074 | 0

6: 0.326176255941391 | 0.46627160906791687 | 0.34321075677871704 | 0.3888437747955322
7: 0.21523083746433258 | 0.6318141222000122 | 0.31386056542396545 | 0.4686185419559479
8: 0.41312989592552185 | 0.45838361978530884 | 0.31495922803878784 | 0.404553085565567
9: 0.2424243539571762 | 0.6368587017059326 | 0.34193798899650574 | 0.4759827256202698
10: 0.19656305015087128 | 0.7578098773956299 | 0.30753204226493835 | 0.560651421546936
11: 0.23480238020420074 | 0.3852837383747101 | 0.27885082364082336 | 0.3129624128341675
12: 0.17843011021614075 | 0.4324907958507538 | 0.3657073974609375 | 0.36106622219085693
13: 0.33237048983573914 | 0.4710817039012909 | 0.25314533710479736 | 0.37523341178894043
14: 0.22131644189357758 | 0.45301318168640137 | 0.31549355387687683 | 0.35737502574920654
15: 0.2892788052558899 | 0.536983072757721 | 0.29984229803085327 | 0.4102093577384949
16: 0.24209225177764893 | 0.7927030324935913 | 0.2563657760620117 | 0.5829715132713318
17: 0.3178254961967468 | 0.575229048728

0: 0.22977861762046814 | 0.259071409702301 | 0.180799201130867 | 0.22788728773593903
1: 0.08466189354658127 | 0.42308613657951355 | 0.1978837549686432 | 0.31932732462882996
2: 0.1336069256067276 | 0.2799014449119568 | 0.20604805648326874 | 0.2237914502620697
3: 0.23516356945037842 | 0.4112149477005005 | 0.1627858728170395 | 0.31007158756256104
4: 0.1316649466753006 | 0.360089510679245 | 0.21405965089797974 | 0.2731904089450836
5: 0.13344088196754456 | 0.28377413749694824 | 0.1766994297504425 | 0.21814072132110596
6: 0.18271462619304657 | 0.15790563821792603 | 0.2058434933423996 | 0.18425807356834412
7: 0.0759592354297638 | 0.2813901901245117 | 0.30604466795921326 | 0.2692420482635498
8: 0.22436270117759705 | 0.2670087218284607 | 0.17111510038375854 | 0.22779731452465057
9: 0.27570948004722595 | 0.34953203797340393 | 0.1890905350446701 | 0.287282794713974
10: 0.18384075164794922 | 0.1495499312877655 | 0.23025332391262054 | 0.19370239973068237
11: 0.19582368433475494 | 0.4406378567218780

46: 0.05031885579228401 | 0.24671176075935364 | 0.10568064451217651 | 0.18516263365745544
47: 0.10214047878980637 | 0.1909591555595398 | 0.09168170392513275 | 0.1437273621559143
48: 0.09435642510652542 | 0.12157008051872253 | 0.0581011064350605 | 0.09874245524406433
49: 0.0996168777346611 | 0.1919224113225937 | 0.1558241844177246 | 0.1587946116924286
# Time passed: 2 s
# Epoch losses | emotion = 0.1253 | speaker = 0.1843 | gender = 0.1288 |
# Train accuracies | emotion = 0.946875 | speaker = 0.94875 | gender = 0.95375 |


In [5]:
ts.execute()

=============TRAINING SESSION STARTED AT EPOCH 1=====================
Epoch #1
# Time passed: 11 s
# Epoch losses | emotion = 0.8703 | speaker = 2.4668 | gender = 0.7733 | total = 1.8151 |
# Train accuracies | emotion = 0.17375704766786262 | speaker = 0.11122501281394157 | gender = 0.49769349051768325 |
# Validation process on validation set
# Validation losses | emotion = 0.0146 | speaker = 0.0402 | gender = 0.0133 | total = 0.0295 |
# Validation accuracies | emotion = 0.17008196721311475 | speaker = 0.125 | gender = 0.5102459016393442 |
# Saving checkpoint...
## Saving best model_alex
# Done and done!
Epoch #2
# Time passed: 11 s
# Epoch losses | emotion = 0.8655 | speaker = 2.3779 | gender = 0.8396 | total = 1.7456 |
# Train accuracies | emotion = 0.1861865709892363 | speaker = 0.13006150691952845 | gender = 0.4546386468477704 |
# Validation process on validation set
# Validation losses | emotion = 0.0135 | speaker = 0.0318 | gender = 0.0146 | total = 0.0235 |
# Validation accuracie

# Validation losses | emotion = 0.0057 | speaker = 0.0075 | gender = 0.0024 | total = 0.0061 |
# Validation accuracies | emotion = 0.8360655737704918 | speaker = 0.7848360655737705 | gender = 0.9426229508196722 |
# Saving checkpoint...
# Done and done!
Epoch #15
# Time passed: 11 s
# Epoch losses | emotion = 0.4166 | speaker = 0.5607 | gender = 0.2308 | total = 0.4644 |
# Train accuracies | emotion = 0.673705791901589 | speaker = 0.5553049718093286 | gender = 0.7788313685289595 |
# Validation process on validation set
# Validation losses | emotion = 0.0042 | speaker = 0.0037 | gender = 0.0022 | total = 0.0036 |
# Validation accuracies | emotion = 0.8381147540983607 | speaker = 0.7950819672131147 | gender = 0.9579918032786885 |
# Saving checkpoint...
## Saving best model_alex
# Done and done!
Epoch #16
# Time passed: 11 s
# Epoch losses | emotion = 0.4027 | speaker = 0.5498 | gender = 0.2191 | total = 0.4529 |
# Train accuracies | emotion = 0.6837038698103537 | speaker = 0.5719662993336

# Validation losses | emotion = 0.0020 | speaker = 0.0220 | gender = 0.0046 | total = 0.0178 |
# Validation accuracies | emotion = 0.8596311475409836 | speaker = 0.8084016393442623 | gender = 0.9497950819672131 |
# Saving checkpoint...
## Saving best model_alex
# Done and done!
Epoch #29
# Time passed: 12 s
# Epoch losses | emotion = 0.3347 | speaker = 0.2997 | gender = 0.1515 | total = 0.3055 |
# Train accuracies | emotion = 0.7565881334063875 | speaker = 0.7061719012354407 | gender = 0.8704908181480762 |
# Validation process on validation set
# Validation losses | emotion = 0.0119 | speaker = 0.0038 | gender = 0.0017 | total = 0.0091 |
# Validation accuracies | emotion = 0.8555327868852459 | speaker = 0.8104508196721312 | gender = 0.9620901639344263 |
# Saving checkpoint...
# Done and done!
Epoch #30
# Time passed: 11 s
# Epoch losses | emotion = 0.3220 | speaker = 0.2725 | gender = 0.1394 | total = 0.2855 |
# Train accuracies | emotion = 0.7601998974884675 | speaker = 0.713087305655

# Validation losses | emotion = 0.0050 | speaker = 0.0079 | gender = 0.0032 | total = 0.0060 |
# Validation accuracies | emotion = 0.8668032786885246 | speaker = 0.8555327868852459 | gender = 0.9661885245901639 |
# Saving checkpoint...
# Done and done!
Epoch #43
# Time passed: 11 s
# Epoch losses | emotion = 0.2463 | speaker = 0.1929 | gender = 0.1068 | total = 0.2200 |
# Train accuracies | emotion = 0.7961570095240366 | speaker = 0.7783903305400928 | gender = 0.9062019477191184 |
# Validation process on validation set
# Validation losses | emotion = 0.0050 | speaker = 0.0080 | gender = 0.0008 | total = 0.0065 |
# Validation accuracies | emotion = 0.8719262295081968 | speaker = 0.8370901639344263 | gender = 0.9672131147540983 |
# Saving checkpoint...
# Done and done!
Epoch #44
# Time passed: 11 s
# Epoch losses | emotion = 0.2447 | speaker = 0.1706 | gender = 0.1010 | total = 0.2100 |
# Train accuracies | emotion = 0.7983435068263361 | speaker = 0.7822095894879083 | gender = 0.90801919

# Validation losses | emotion = 0.0157 | speaker = 0.0372 | gender = 0.0014 | total = 0.0300 |
# Validation accuracies | emotion = 0.8555327868852459 | speaker = 0.8555327868852459 | gender = 0.9610655737704918 |
# Saving checkpoint...
# Done and done!
Epoch #57
# Time passed: 11 s
# Epoch losses | emotion = 0.1627 | speaker = 0.1091 | gender = 0.0680 | total = 0.1430 |
# Train accuracies | emotion = 0.824570395748469 | speaker = 0.8210274533077954 | gender = 0.9260118517719208 |
# Validation process on validation set
# Validation losses | emotion = 0.0129 | speaker = 0.0047 | gender = 0.0016 | total = 0.0099 |
# Validation accuracies | emotion = 0.8616803278688525 | speaker = 0.8575819672131147 | gender = 0.9631147540983607 |
# Saving checkpoint...
# Done and done!
Epoch #58
# Time passed: 12 s
# Epoch losses | emotion = 0.1599 | speaker = 0.1341 | gender = 0.0710 | total = 0.1506 |
# Train accuracies | emotion = 0.8264815567613425 | speaker = 0.8233620247795118 | gender = 0.927070998

# Validation losses | emotion = 0.0046 | speaker = 0.0066 | gender = 0.0005 | total = 0.0056 |
# Validation accuracies | emotion = 0.8729508196721312 | speaker = 0.8627049180327869 | gender = 0.9631147540983607 |
# Saving checkpoint...
# Done and done!
Epoch #71
# Time passed: 12 s
# Epoch losses | emotion = 0.1152 | speaker = 0.1285 | gender = 0.0582 | total = 0.1306 |
# Train accuracies | emotion = 0.8486402783693447 | speaker = 0.8497917283300005 | gender = 0.9388684748160928 |
# Validation process on validation set
# Validation losses | emotion = 0.0117 | speaker = 0.0142 | gender = 0.0034 | total = 0.0120 |
# Validation accuracies | emotion = 0.8586065573770492 | speaker = 0.8596311475409836 | gender = 0.9631147540983607 |
# Saving checkpoint...
# Done and done!
Epoch #72
# Time passed: 12 s
# Epoch losses | emotion = 0.1037 | speaker = 0.0942 | gender = 0.0478 | total = 0.1056 |
# Train accuracies | emotion = 0.8501836664958141 | speaker = 0.8515255709322854 | gender = 0.93961430

# Validation losses | emotion = 0.0084 | speaker = 0.0025 | gender = 0.0013 | total = 0.0065 |
# Validation accuracies | emotion = 0.8278688524590164 | speaker = 0.8504098360655737 | gender = 0.9610655737704918 |
# Saving checkpoint...
# Done and done!
Epoch #85
# Time passed: 11 s
# Epoch losses | emotion = 0.0714 | speaker = 0.0615 | gender = 0.0316 | total = 0.0762 |
# Train accuracies | emotion = 0.867962131033859 | speaker = 0.8707992884493623 | gender = 0.9478849458799409 |
# Validation process on validation set
# Validation losses | emotion = 0.0024 | speaker = 0.0073 | gender = 0.0005 | total = 0.0058 |
# Validation accuracies | emotion = 0.8657786885245902 | speaker = 0.860655737704918 | gender = 0.9672131147540983 |
# Saving checkpoint...
# Done and done!
Epoch #86
# Time passed: 12 s
# Epoch losses | emotion = 0.0767 | speaker = 0.0615 | gender = 0.0316 | total = 0.0791 |
# Train accuracies | emotion = 0.8691487966814871 | speaker = 0.8721079231878702 | gender = 0.9484313351

# Validation losses | emotion = 0.0018 | speaker = 0.0083 | gender = 0.0009 | total = 0.0066 |
# Validation accuracies | emotion = 0.8340163934426229 | speaker = 0.8504098360655737 | gender = 0.9631147540983607 |
# Saving checkpoint...
# Done and done!
Epoch #99
# Time passed: 12 s
# Epoch losses | emotion = 0.0562 | speaker = 0.0648 | gender = 0.0283 | total = 0.0723 |
# Train accuracies | emotion = 0.8832947620748748 | speaker = 0.8865849680816364 | gender = 0.9544962697192323 |
# Validation process on validation set
# Validation losses | emotion = 0.0119 | speaker = 0.0057 | gender = 0.0002 | total = 0.0097 |
# Validation accuracies | emotion = 0.8575819672131147 | speaker = 0.8545081967213115 | gender = 0.9620901639344263 |
# Saving checkpoint...
# Done and done!
Epoch #100
# Time passed: 11 s
# Epoch losses | emotion = 0.0559 | speaker = 0.0464 | gender = 0.0253 | total = 0.0603 |
# Train accuracies | emotion = 0.8842696053305997 | speaker = 0.8875858534085085 | gender = 0.9549154

# Validation losses | emotion = 0.0107 | speaker = 0.0216 | gender = 0.0029 | total = 0.0167 |
# Validation accuracies | emotion = 0.8452868852459017 | speaker = 0.8637295081967213 | gender = 0.9661885245901639 |
# Saving checkpoint...
# Done and done!
Epoch #113
# Time passed: 11 s
# Epoch losses | emotion = 0.0467 | speaker = 0.0506 | gender = 0.0185 | total = 0.0606 |
# Train accuracies | emotion = 0.8955901897370534 | speaker = 0.8988310963744484 | gender = 0.9596462898536262 |
# Validation process on validation set
# Validation losses | emotion = 0.0157 | speaker = 0.0093 | gender = 0.0053 | total = 0.0119 |
# Validation accuracies | emotion = 0.8647540983606558 | speaker = 0.8698770491803278 | gender = 0.9682377049180327 |
# Saving checkpoint...
# Done and done!
Epoch #114
# Time passed: 12 s
# Epoch losses | emotion = 0.0368 | speaker = 0.0370 | gender = 0.0188 | total = 0.0456 |
# Train accuracies | emotion = 0.8964071506290072 | speaker = 0.8996151321409623 | gender = 0.959973

KeyboardInterrupt: 

In [4]:
ts.results_dict

{'emotion': {'train accuracy': [0.17375704766786262,
   0.1861865709892363,
   0.22133948402528617,
   0.26736289082521786,
   0.3648385443362378,
   0.4406287373996241,
   0.49589953869810355,
   0.5374487442337262,
   0.5696508912808247,
   0.5954638646847771,
   0.6166068682726806,
   0.6341192550828635,
   0.6491937073690021,
   0.6622061946254668,
   0.673705791901589,
   0.6837038698103537,
   0.6927216811891338,
   0.7007090381001196,
   0.7077207370039655,
   0.7145566376217325,
   0.7206682775621781,
   0.7264805927030428,
   0.7316203507677222,
   0.7363851870835469,
   0.7409226037929267,
   0.7451504159602571,
   0.7492454012187483,
   0.7529838178223621,
   0.7565881334063875,
   0.7601998974884675,
   0.7636530480646815,
   0.7668423244490005,
   0.7699470357081838,
   0.7729444930201707,
   0.7756754777769642,
   0.7784184748561991,
   0.7811794367406873,
   0.7838355499204187,
   0.7863554521678561,
   0.7889479753972322,
   0.7913827805628133,
   0.7938969026872666,
  

In [5]:
ts.train_dataset.tasks = ('emotion', 'speaker', 'gender')

In [6]:
ts.create_results_file()

In [25]:
results = pd.read_csv(os.path.join(ts.path_to_results, f'{ts.name}_results.csv'), delimiter=';')
results

,result,subset,metric,epoch,task
0,0.173757,train,accuracy,1,emotion
1,0.186187,train,accuracy,2,emotion
2,0.221339,train,accuracy,3,emotion
3,0.267363,train,accuracy,4,emotion
4,0.364839,train,accuracy,5,emotion
...,...,...,...,...,...
1411,0.000158,test,loss,114,gender
1412,0.000501,test,loss,115,gender
1413,0.001041,test,loss,116,gender
1414,0.000264,test,loss,117,gender


In [26]:
test_loss = results.loc[(results['subset'] == 'test') & (results['metric'] == 'loss')]
test_loss

,result,subset,metric,epoch,task
354,0.014558,test,loss,1,emotion
355,0.013541,test,loss,2,emotion
356,0.011909,test,loss,3,emotion
357,0.011126,test,loss,4,emotion
358,0.008335,test,loss,5,emotion
...,...,...,...,...,...
1411,0.000158,test,loss,114,gender
1412,0.000501,test,loss,115,gender
1413,0.001041,test,loss,116,gender
1414,0.000264,test,loss,117,gender


In [27]:
emotion_results = test_loss.loc[test_loss['task'] == 'emotion']
speaker_results = test_loss.loc[test_loss['task'] == 'speaker']
gender_results = test_loss.loc[test_loss['task'] == 'gender']

In [28]:
emotion_results

,result,subset,metric,epoch,task
354,0.014558,test,loss,1,emotion
355,0.013541,test,loss,2,emotion
356,0.011909,test,loss,3,emotion
357,0.011126,test,loss,4,emotion
358,0.008335,test,loss,5,emotion
...,...,...,...,...,...
467,0.026891,test,loss,114,emotion
468,0.003300,test,loss,115,emotion
469,0.004641,test,loss,116,emotion
470,0.002686,test,loss,117,emotion


In [29]:
emotion_results.loc[emotion_results['result'] == emotion_results['result'].min()]

,result,subset,metric,epoch,task
411,0.000568,test,loss,58,emotion


In [30]:
best_epoch = results.loc[results['epoch'] == 58]
best_epoch = best_epoch.drop('epoch', axis=1)
best_epoch

,result,subset,metric,task
57,0.826482,train,accuracy,emotion
175,0.868852,test,accuracy,emotion
293,0.159852,train,loss,emotion
411,0.000568,test,loss,emotion
529,0.823362,train,accuracy,speaker
647,0.846311,test,accuracy,speaker
765,0.134055,train,loss,speaker
883,0.022737,test,loss,speaker
1001,0.927071,train,accuracy,gender
1119,0.968238,test,accuracy,gender


## Вторая модель (на основе VGG)

In [2]:
model = md_multi.vgg(num_emotions=2, num_speakers=12, num_genders=2, type=11, bn=True)
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 
device = torch.device("cpu")
model = model.to(device)
print(model)
summary(model, (1, 224, 224), batch_size=32,
        device='cpu'
       )
model=None
torch.cuda.empty_cache()

VGG(
  (features): Sequential(
    (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU(inplace=True)
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU(inplace=True)
    (11): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (13): ReLU(inplace=True)
    (14): MaxPool2d(ke

In [5]:
train_ds = ramas_224_train
test_ds = ramas_224_test
model = md_multi.vgg(num_emotions=2, num_speakers=12, num_genders=2, type=11, bn=True)
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu") 
# device = torch.device("cpu") 
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model.parameters(), lr=1e-4)
ts = TrainingSession(name='VGGAverageWeighting',
                      model=model,
                      train_dataset=train_ds,
                      test_dataset=test_ds,
                      criterion=criterion,
                      optimizer=optimizer,
                      num_epochs=300,
                      batch_size=32,
                      device=device,
                     path_to_weights=WEIGHTS_FOLDER,
                     path_to_results=RESULTS_FOLDER,
                    loss_weighter=averaged_sum  # Из файла multitask_training_session.py
                    )

INITIALIZING TRAINING SESSION...
Loaders ready
TRAINING SESSION VGGAverageWeighting__Ramas224BinaryMulti_224_train INITIALIZED
Trying to load checkpoint from file
Found file
Loading file models/training_sessions/VGGAverageWeighting__Ramas224BinaryMulti_224_train.pt
Updating model...
Updating optimizer...
Success!


In [4]:
ts.execute()

=============TRAINING SESSION STARTED AT EPOCH 1=====================
Epoch #1
# Time passed: 25 s
# Epoch losses | emotion = 0.7359 | speaker = 2.0338 | gender = 0.7283 | total = 1.4965 |
# Train accuracies | emotion = 0.4887237314197847 | speaker = 0.2780625320348539 | gender = 0.5366478728856996 |
# Validation process on validation set
# Validation losses | emotion = 0.0061 | speaker = 0.0317 | gender = 0.0087 | total = 0.0240 |
# Validation accuracies | emotion = 0.8299180327868853 | speaker = 0.45491803278688525 | gender = 0.7899590163934426 |
# Saving checkpoint...
## Saving best model_alex
# Done and done!
Epoch #2
# Time passed: 25 s
# Epoch losses | emotion = 0.4833 | speaker = 1.1011 | gender = 0.4037 | total = 0.8210 |
# Train accuracies | emotion = 0.6515889287544849 | speaker = 0.4429779600205023 | gender = 0.6979753972321886 |
# Validation process on validation set
# Validation losses | emotion = 0.0069 | speaker = 0.0211 | gender = 0.0062 | total = 0.0155 |
# Validation 

# Validation losses | emotion = 0.0052 | speaker = 0.0218 | gender = 0.0056 | total = 0.0164 |
# Validation accuracies | emotion = 0.7940573770491803 | speaker = 0.8913934426229508 | gender = 0.9661885245901639 |
# Saving checkpoint...
# Done and done!
Epoch #15
# Time passed: 25 s
# Epoch losses | emotion = 0.2904 | speaker = 0.2200 | gender = 0.1209 | total = 0.2538 |
# Train accuracies | emotion = 0.8257303946694003 | speaker = 0.8177686656415514 | gender = 0.9242952332137365 |
# Validation process on validation set
# Validation losses | emotion = 0.0064 | speaker = 0.0087 | gender = 0.0025 | total = 0.0070 |
# Validation accuracies | emotion = 0.882172131147541 | speaker = 0.8514344262295082 | gender = 0.9661885245901639 |
# Saving checkpoint...
# Done and done!
Epoch #16
# Time passed: 25 s
# Epoch losses | emotion = 0.2765 | speaker = 0.1774 | gender = 0.1104 | total = 0.2297 |
# Train accuracies | emotion = 0.8295265248590467 | speaker = 0.8255061506919529 | gender = 0.927889543

# Validation losses | emotion = 0.0045 | speaker = 0.0016 | gender = 0.0012 | total = 0.0033 |
# Validation accuracies | emotion = 0.8545081967213115 | speaker = 0.8811475409836066 | gender = 0.9733606557377049 |
# Saving checkpoint...
# Done and done!
Epoch #29
# Time passed: 25 s
# Epoch losses | emotion = 0.1225 | speaker = 0.1116 | gender = 0.0594 | total = 0.1242 |
# Train accuracies | emotion = 0.8718517471146539 | speaker = 0.8833842945262377 | gender = 0.952376323370862 |
# Validation process on validation set
# Validation losses | emotion = 0.0089 | speaker = 0.0027 | gender = 0.0007 | total = 0.0071 |
# Validation accuracies | emotion = 0.7807377049180327 | speaker = 0.8647540983606558 | gender = 0.9764344262295082 |
# Saving checkpoint...
# Done and done!
Epoch #30
# Time passed: 25 s
# Epoch losses | emotion = 0.1101 | speaker = 0.1016 | gender = 0.0532 | total = 0.1139 |
# Train accuracies | emotion = 0.8747479924824876 | speaker = 0.8862634546386469 | gender = 0.953579361

# Validation losses | emotion = 0.0032 | speaker = 0.0147 | gender = 0.0022 | total = 0.0115 |
# Validation accuracies | emotion = 0.8770491803278688 | speaker = 0.9211065573770492 | gender = 0.9805327868852459 |
# Saving checkpoint...
# Done and done!
Epoch #43
# Time passed: 25 s
# Epoch losses | emotion = 0.0541 | speaker = 0.0441 | gender = 0.0266 | total = 0.0582 |
# Train accuracies | emotion = 0.9033828805740646 | speaker = 0.9134731145625976 | gender = 0.9647527207275934 |
# Validation process on validation set
# Validation losses | emotion = 0.0006 | speaker = 0.0002 | gender = 0.0003 | total = 0.0004 |
# Validation accuracies | emotion = 0.8811475409836066 | speaker = 0.9190573770491803 | gender = 0.9795081967213115 |
# Saving checkpoint...
# Done and done!
Epoch #44
# Time passed: 25 s
# Epoch losses | emotion = 0.0579 | speaker = 0.0744 | gender = 0.0298 | total = 0.0805 |
# Train accuracies | emotion = 0.9051302362424863 | speaker = 0.9148979544289642 | gender = 0.96541400

# Validation losses | emotion = 0.0063 | speaker = 0.0125 | gender = 0.0030 | total = 0.0094 |
# Validation accuracies | emotion = 0.8627049180327869 | speaker = 0.9088114754098361 | gender = 0.9815573770491803 |
# Saving checkpoint...
# Done and done!
Epoch #57
# Time passed: 25 s
# Epoch losses | emotion = 0.0445 | speaker = 0.0438 | gender = 0.0231 | total = 0.0560 |
# Train accuracies | emotion = 0.9232062729864128 | speaker = 0.9309036301671657 | gender = 0.972236459935076 |
# Validation process on validation set
# Validation losses | emotion = 0.0086 | speaker = 0.0086 | gender = 0.0000 | total = 0.0086 |
# Validation accuracies | emotion = 0.8668032786885246 | speaker = 0.8934426229508197 | gender = 0.9805327868852459 |
# Saving checkpoint...
# Done and done!
Epoch #58
# Time passed: 25 s
# Epoch losses | emotion = 0.0432 | speaker = 0.0401 | gender = 0.0182 | total = 0.0528 |
# Train accuracies | emotion = 0.9242651867300589 | speaker = 0.9318784354619205 | gender = 0.972666537

# Validation losses | emotion = 0.0000 | speaker = 0.0006 | gender = 0.0000 | total = 0.0005 |
# Validation accuracies | emotion = 0.8811475409836066 | speaker = 0.9149590163934426 | gender = 0.9784836065573771 |
# Saving checkpoint...
## Saving best model_alex
# Done and done!
Epoch #71
# Time passed: 25 s
# Epoch losses | emotion = 0.0408 | speaker = 0.0476 | gender = 0.0158 | total = 0.0605 |
# Train accuracies | emotion = 0.9360746745980754 | speaker = 0.9421820518188578 | gender = 0.9769493434208532 |
# Validation process on validation set
# Validation losses | emotion = 0.0140 | speaker = 0.0002 | gender = 0.0000 | total = 0.0138 |
# Validation accuracies | emotion = 0.8709016393442623 | speaker = 0.9241803278688525 | gender = 0.9764344262295082 |
# Saving checkpoint...
# Done and done!
Epoch #72
# Time passed: 25 s
# Epoch losses | emotion = 0.0287 | speaker = 0.0309 | gender = 0.0141 | total = 0.0381 |
# Train accuracies | emotion = 0.9368308274958711 | speaker = 0.942817785750

# Validation losses | emotion = 0.0055 | speaker = 0.0063 | gender = 0.0006 | total = 0.0057 |
# Validation accuracies | emotion = 0.8790983606557377 | speaker = 0.9272540983606558 | gender = 0.9825819672131147 |
# Saving checkpoint...
# Done and done!
Epoch #85
# Time passed: 25 s
# Epoch losses | emotion = 0.0200 | speaker = 0.0197 | gender = 0.0081 | total = 0.0296 |
# Train accuracies | emotion = 0.9450960291856363 | speaker = 0.9501130641903096 | gender = 0.9803599963819459 |
# Validation process on validation set
# Validation losses | emotion = 0.0053 | speaker = 0.0009 | gender = 0.0000 | total = 0.0047 |
# Validation accuracies | emotion = 0.8698770491803278 | speaker = 0.9221311475409836 | gender = 0.9866803278688525 |
# Saving checkpoint...
# Done and done!
Epoch #86


KeyboardInterrupt: 

In [6]:
ts.train_dataset.tasks = ('emotion', 'speaker', 'gender')
ts.create_results_file()


In [7]:
results = pd.read_csv(os.path.join(ts.path_to_results, f'{ts.name}_results.csv'), delimiter=';')
results

,result,subset,metric,epoch,task
0,0.488724,train,accuracy,1,emotion
1,0.651589,train,accuracy,2,emotion
2,0.710490,train,accuracy,3,emotion
3,0.739493,train,accuracy,4,emotion
4,0.757150,train,accuracy,5,emotion
...,...,...,...,...,...
1015,0.000092,test,loss,81,gender
1016,0.000084,test,loss,82,gender
1017,0.001346,test,loss,83,gender
1018,0.000586,test,loss,84,gender


In [8]:
test_loss = results.loc[(results['subset'] == 'test') & (results['metric'] == 'loss')]
emotion_results = test_loss.loc[test_loss['task'] == 'emotion']
speaker_results = test_loss.loc[test_loss['task'] == 'speaker']
gender_results = test_loss.loc[test_loss['task'] == 'gender']
emotion_results

,result,subset,metric,epoch,task
255,0.006106,test,loss,1,emotion
256,0.006918,test,loss,2,emotion
257,0.008478,test,loss,3,emotion
258,0.008317,test,loss,4,emotion
259,0.006289,test,loss,5,emotion
...,...,...,...,...,...
335,0.002679,test,loss,81,emotion
336,0.024332,test,loss,82,emotion
337,0.004888,test,loss,83,emotion
338,0.005468,test,loss,84,emotion


In [9]:
emotion_results.loc[emotion_results['result'] == emotion_results['result'].min()]

,result,subset,metric,epoch,task
324,0.000045,test,loss,70,emotion


In [10]:
best_epoch = results.loc[results['epoch'] == 58]
best_epoch = best_epoch.drop('epoch', axis=1)
best_epoch

,result,subset,metric,task
57,0.924265,train,accuracy,emotion
142,0.872951,test,accuracy,emotion
227,0.043218,train,loss,emotion
312,0.002622,test,loss,emotion
397,0.931878,train,accuracy,speaker
482,0.917008,test,accuracy,speaker
567,0.040145,train,loss,speaker
652,0.000083,test,loss,speaker
737,0.972667,train,accuracy,gender
822,0.983607,test,accuracy,gender
